**RUNNING AND OPTIMIZING PRO_ANTI**

The definition of the ProAnti network, as well functions to run it and evaluate the cost of a set of runs, get extracted into

    pro_anti.jl
    
At that point we're reaching current development as opposed to stable files.
One cell, an example of running farm F, gets extracted into

    farming.jl
    
    

<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [19]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('make_table_of_contents.js')
"""

# ProAnti

In [2]:
#@include_me pro_anti.jl

include("rate_networks.jl")  # that will also include general_utils.jl, constrained_parabolic_minimization.jl, and hessian_utils.jl


backwardsModel

In [77]:
#@   no longer include_me  pro_anti.jl

#  OLD VERSION OF PLOT_PA -- SUPERSEDED BY CELL BELOW 13-Dec-2017
"""
    plot_PA(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, post_target_period=1,
        plot_Us=true, ax_set=Dict(), other_unused_params...)

Helper function for plotting ProAnti results. Given a time axis and nunits-by-nsteps U and V matrices, will 
plot them (first clearing the figure if clearfig=true but overlaying if clearfig=false) in three vertically-
arranged subplots. The top one has V as a function of time, with the two Pro units in blues and the two Anti 
units in reds, dark colors for the left side of the brain, light colors for the right.  The middle subplot
has Us. And the bottom subplot shows the difference between the two Pro units as a function of time.

# PARAMETERS:

- t     a vector representing the time axis

- U     a 4-by-length(t) matrix representing activities of Pro_L, Anti_L, Anti_R, Pro_R units

- V     as with U but for the unit outputs, after going through the sigmoid. We expect 0<=V<=1.

# OPTIONAL PARAMETERS:

- fignum   Which figure to plot on

- clearfig   If ax_set is empty and clearfig=true, clears the figure before doing anything

- rule_and_delay_period   duration of the rule_and_delay_period; a vertical line will be placed at its end

- target_period   duration of the target period; a vertical line will be placed at its end

- post_target_period   duration of the post_target period; a vertical line will be placed at its end

- plot_Us    If true, makes an axis for, and plots, U.  If false ignores U.

- ax_set     A dictionary. If it has an enty with key "Vax", the value should be an 
             axis handle, on which V will be plotted. Value for key "Uax" will be axis where U should be plotted;
             and value for key "Dax" will be axis where the difference is plotted. 
             If ax_set is not empty, clearfig is ignored and the figure is not cleared beforehand.


"""
function plot_PA_old(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, post_target_period=1,
    plot_Us=true, ax_set=Dict(), other_unused_params...)
    figure(fignum)
    if isempty(ax_set) && clearfig; clf(); end
    
    if ~haskey(ax_set, "Vax")
        if plot_Us; ax1 = subplot(3,1,1); else ax1=subplot(2,1,1); end;
    else
        ax1 = safe_axes(ax_set["Vax"], fontsize=20)
    end
    h = plot(t, V'); 
    setp(h[1], color=[0, 0, 1])
    setp(h[2], color=[1, 0, 0])
    setp(h[3], color=[1, 0.5, 0.5])
    setp(h[4], color=[0, 1, 1])
    ylabel("V")

    ax = gca()
    oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
    ylim(minimum([V[:];oldlims[1]])-0.1, maximum([V[:];oldlims[2]])+0.1)
    yl = [ylim()[1], ylim()[2]]
    vlines([rule_and_delay_period, 
            rule_and_delay_period+target_period,
            rule_and_delay_period+target_period+post_target_period], 
            -0.05, 1.05, linewidth=2)
    if yl[1]<0.02
        yl[1] = -0.02
    end
    if yl[2]>0.98
        yl[2] = 1.02
    end
    ylim(yl)
    grid(true)
    remove_xtick_labels(ax1)
        
    if plot_Us
        if ~haskey(ax_set, "Uax")
            ax2 = subplot(3,1,2)
        else
            ax2 = safe_axes(ax_set["Uax"])
        end
        hu = plot(t, U')
        oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
        ylim(minimum([U[:];oldlims[1]])-0.1, maximum([U[:];oldlims[2]])+0.1)
        setp(hu[1], color=[0, 0, 1])
        setp(hu[2], color=[1, 0, 0])
        setp(hu[3], color=[1, 0.5, 0.5])
        setp(hu[4], color=[0, 1, 1])
        ylabel("U"); 
        vlines([rule_and_delay_period, 
                rule_and_delay_period+target_period,
                rule_and_delay_period+target_period+post_target_period], 
                ylim()[1], ylim()[2], linewidth=2)
        remove_xtick_labels(ax2)

        grid(true)
    end
    
    if ~haskey(ax_set, "Dax")
        if plot_Us; ax3 = subplot(3,1,3); else ax3=subplot(2,1,2); end;
    else
        ax3 = safe_axes(ax_set["Dax"])
    end

    delta = V[1,:] - V[4,:]
    hr = plot(t, delta, "g")
    oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
    ylim(minimum([delta[:];oldlims[1]])-0.1, maximum([delta[:];oldlims[2]])+0.1)
    vlines([rule_and_delay_period, 
            rule_and_delay_period+target_period,
            rule_and_delay_period+target_period+post_target_period], 
            ylim()[1], ylim()[2], linewidth=2)
    hlines(0, t[1], t[end], linewidth=2)
    xlabel("t"); ylabel("Pro R - Pro L")
    grid(true)
        
end



plot_PA_old

In [79]:
#@include_me  pro_anti.jl

"""
    plot_PA(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, 
        post_target_period=1, fontsize=20, color_list = [0 0 1; 1 0 0 ; 1 0.5 0.5 ; 0 1 1],
        singleton_color = [0 0.5 0],
        plottables = ["V", "U", "V[1,:]-V[4,:]"], ylabels = ["V", "U", "Pro_R - Pro_L"],
        ylims = [[-0.02, 1.02], nothing, [-1.02, 1.02]], plot_Us = nothing,
        ax_set=nothing, other_unused_params...)

Helper function for plotting ProAnti results. Given a time axis and nunits-by-ntimesteps 
U and V matrices, will plot an arbitrary number of functions of them. Default is to 
plot V vs time, U versus time, and V[1,:]-V[4,:] versus time in three vertically
arranged subplots.

The things to plot are determined by the three optional parameters `plottables`,
`ylabels`, and `ylims'

# PARAMETERS:

- t     a vector representing the time axis

- U     a 4-by-length(t) matrix representing activities of Pro_L, Anti_L, Anti_R, Pro_R units

- V     as with U but for the unit outputs, after going through the sigmoid. We expect 0<=V<=1.

# OPTIONAL PARAMETERS:

- plottables   A vector of strings. Each of these strings indicates something to 
            plot; the strings will be evaluated in a context where the variables "V", "U", 
            and "t" are instantiated to have the values passed to `plot_PA()`. Thus 
            plottables=["V", "V[1,:]-V[4,:]"] indicates that two axes should be used, on
            the first one V will be plotted, on the second V[1,:]-V[4,:].
                The strings can be any arbitrary Julia expression, with the condition that
            their evaluation should produce something with either nrows or ncolumns
            equal to length(t).

- ylabels   A vector of strings, should be equal in length to plottables; Each element
            here will be used as the y label for the corresponding axis.

- ylims     A vector of Any, should be in length to plottables. If an element is
            `nothing`, then allow automatic scaling of the y axes for this axis. Otherwise,
            the element should be a 2-long vector of Float64, indicating the minimum and
            the maximum for the y axis, respectively.

- ax_set    If passed, should be a vector of axis handles, equal in length to 
            plottables. If not passed, new axes will be created, vertically stacked, number 
            equal to length of plottables.

            Finally, for backwards compatibility, ax_set may also be a Dict, in which case it is 
            expected to have keys "Vax", "Uax", and "Dax", each with value equal to an axis handle, 
            on which V, U, and V[1,:] - V[4,:] will be plotted. In this backwards compatibility
            mode, if plot_Us=false, then U is not plotted and "Uax" is not needed.

- plot_Us  If passed, should be either true or false, and indicates that we are in 
            backwards compatibility mode (see ax_set above).

- singleton_color   If the result of evaluating plottables[i] has one dimension
            of length 1 (i.e., it is a vector, not a matrix) then that line will be plotted in
            this color.

- color_list If all dimensions of the result of evaluating plottables[i] are bigger than 1,
            then the color order of the resulting lines plotted will be the rows of color_list

- fignum   Which figure to plot on; only used if ax_set is empty

- clearfig   If ax_set is empty and clearfig=true, clears the figure before doing anything

- rule_and_delay_period   duration of the rule_and_delay_period; a vertical line will be placed at its end

- target_period   duration of the target period; a vertical line will be placed at its end

- post_target_period   duration of the post_target period; a vertical line will be placed at its end


"""
function plot_PA(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, 
    post_target_period=1, fontsize=20, color_list = [0 0 1; 1 0 0 ; 1 0.5 0.5 ; 0 1 1],
    singleton_color = [0 0.5 0],
    plottables = ["V", "U", "V[1,:]-V[4,:]"], ylabels = ["V", "U", "Pro_R - Pro_L"],
    ylims = [[-0.02, 1.02], nothing, [-1.02, 1.02]], plot_Us = nothing,
    ax_set=nothing, other_unused_params...)
    
    if plot_Us != nothing || typeof(ax_set)<:Dict
    # Backwards compatibility mode!
        if plot_Us==true || plot_Us==nothing
            plottables = ["V", "U", "V[1,:]-V[4,:]"]
            ylabels    = ["V", "U", "Pro_R - Pro_L"]
            ylims      = [[-0.02, 1.02], nothing, [-1.02, 1.02]]
            if typeof(ax_set)<:Dict
                ax_set = [ax_set["Vax"], ax_set["Uax"], ax_set["Dax"]]
            end
        else
            plottables = ["V", "V[1,:]-V[4,:]"]
            ylabels    = ["V", "Pro_R - Pro_L"]
            ylims      = [[-0.02, 1.02], [-1.02, 1.02]]
            if typeof(ax_set)<:Dict
                ax_set = [ax_set["Vax"], ax_set["Dax"]]
            end
        end
    end
    
    if ax_set==nothing
        figure(fignum)
        if clearfig; clf(); end
    end
        
    nplots = length(plottables)
    if length(ylabels) != nplots
        error("Must have as many entries in ylabels as things that will be plotted")
    end
    if ax_set==nothing
        ax_set = Array{PyCall.PyObject}(nplots,1)
        for i=1:nplots
            ax_set[i] = subplot(nplots,1,i)
            ax_set[i] = safe_axes(ax_set[i], fontsize=20)
        end
    end
    if length(ax_set) != nplots
        error("Must have as many entries in ax_set as things that will be plotted")
    end

    vardict = Dict("V"=>V, "U"=>U, "t"=>t)
    
    for i=1:nplots
        safe_axes(ax_set[i])
        plotvals = replacer(plottables[i], vardict)
        if size(plotvals,1) != length(t); plotvals = plotvals'; end
        h = plot(t, plotvals)
        if length(h)==1
            setp(h, color=singleton_color[:])
        else
            for j=1:length(h); setp(h[j], color=color_list[j,:]); end
        end
        ylabel(ylabels[i])

        if ylims[i]==nothing
            oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
            ylim(minimum([plotvals[:];oldlims[1]])-0.1, maximum([plotvals[:];oldlims[2]])+0.1)
        else
            ylim(ylims[i][1], ylims[i][2])
        end
        vlines([rule_and_delay_period, 
                rule_and_delay_period+target_period,
                rule_and_delay_period+target_period+post_target_period], 
                ylim()[1], ylim()[2], linewidth=2)
        xlim((t[1]-0.01, t[end]+0.01))
        grid(true)
        if i<nplots; remove_xtick_labels(ax_set[i]); else xlabel("t"); end
    end
end



plot_PA

### testing plot_PA()

In [81]:
pygui(true); figure(1); clf();
Vax = subplot(3,4,1)
Uax = subplot(3,4,6)
Dax = subplot(3,4,11)


t = 0:0.1:10;
U = randn(4, length(t))
V = randn(4, length(t))
plot_PA(t, U, V, fignum=1, ax_set=Dict("Vax"=>Vax, "Uax"=>Uax, "Dax"=>Dax))

QPixmap::scaled: Pixmap is a null pixmap


In [5]:
# pygui(true)
# pro_ax_set, anti_ax_set = plot_farm2("farm_LD0032");

# plot_PA(0, 0, 0; ax_set=pro_ax_set, clearfig=false)
# axes(pro_ax_set["Vax"])

### Parsing symbols in opto times

In [6]:
#@include_me   pro_anti.jl

"""
parsed_times = parse_opto_times(opto_times, model_params)

This function takes period specified with symbols and turns them into times in secs. Each element in the 
opto_times parameter can be a number (time in secs) or a string. This string will be parsed and
evaluated. Four special symbols are available: trial_start (which is really zero), target_start, 
target_end, and trial_end.  To compute these, IT IS ASSUMED that model_params will contain entries
for :rule_and_delay_period, :target period, and :post_target_period

In addition, for backwards compatibility with Alex's code, some numbers are special:
20 means end of trial (but we prefer the string "trial_end"), 100 mens end of rule and delay (but we prefer
"target_start", and 200 means end of target (we prfer "target_end").

# PARAMETERS

* opto_times, an n-by-2 matrix where each row of the matrix specifies af starting time and an ending time. Entries can be numbers or strings representing expressions to be evaluated.

* model_params  a Dict which must contain key-value pairs with the keys :rule_and_delay_period, :target_period, and :post_target_period

# RETURNS

* a matrix the same size as opto_times, but with all strings parsed and evaluated.

# EXAMPLE

> model_params[:target_period] = 1  ; model_params[:rule_and_delay_period] = 1
> opto_times = ["exp(target_end)" 3; 6 8]   # NOTE: exp() that means the exponential-- any arbitrary Julia expression is allowed
> parse_opto_times(opto_times, model_params)

7.38906   3
 6         8

"""
function parse_opto_times(opto_times2, model_params)
    # Make a copy so we don't futz with the original:
    opto_times = copy(opto_times2)
    
    # we want to be able to stash floats and numbers in by the end:
    if typeof(opto_times)<:Array{String}  ||  typeof(opto_times)<:Array{Int64}
        opto_times = convert(Array{Any}, opto_times)
    end

    
    # Let's define the time markers
    trial_start = target_start = target_end = trial_end = 0   # just define them here so vars are available ourside try/catch
    try
        trial_start  = 0
        target_start = model_params[:rule_and_delay_period]
        target_end   = target_start + model_params[:target_period]
        trial_end    = target_end   + model_params[:post_target_period]
    catch y
        @printf("\n\nwhoa, are you sure your model_params had all three of :rule_and_delay_period, :target_period, and :post_target_period?\n\n")
        error(y)
    end
    
    function replacer(P)   # run through an expression tree, replacing known symbols with their values, then evaluate
        if typeof(P)<:Symbol
            if P == :trial_start;  P = trial_start; end;                    
            if P == :target_start; P = target_start; end;                    
            if P == :target_end;   P = target_end; end;                    
            if P == :trial_end;    P = trial_end; end;                    
            return P
        end        
        for i=1:length(P.args)
            if typeof(P.args[i])<:Expr
                P.args[i] = replacer(P.args[i])
            else
                if P.args[i] == :trial_start;  P.args[i] = trial_start; end;                    
                if P.args[i] == :target_start || P.args[i]==100;  P.args[i] = target_start; end;                    
                if P.args[i] == :target_end   || P.args[i]==200;  P.args[i] = target_end; end;                    
                if P.args[i] == :trial_end    || P.args[i]==20;   P.args[i] = trial_end; end;                    
            end
        end
        return eval(P)
    end
    
    for i=1:length(opto_times)
        if typeof(opto_times[i])==String
            # @printf("Got a string, and it is: %s\n", opto_times[i])
            # @printf("Parsing turned it into: ");  print(parse(opto_times[i])); print("\n")
            # @printf("Replacer turned it into: "); print(replacer(parse(opto_times[i]))); print("\n")
            opto_times[i] = replacer(parse(opto_times[i]))
        elseif typeof(opto_times[i])==Int64
            if opto_times[i]==100; opto_times[i] = target_start; end;
            if opto_times[i]==200; opto_times[i] = target_end;   end;
            if opto_times[i]==20;  opto_times[i] = trial_end;    end;
        end            
    end
    return opto_times
end

parse_opto_times

##### Example of parsing symbols in opto times

In [7]:

mp = Dict(:target_period=>0.5, :rule_and_delay_period=>0.5, :post_target_period=>0.5)

opto_times = ["exp(target_end)" 3; 6 8; 9 10]

parse_opto_times(opto_times, mp)

# Or, using, Alex's special time codes:

# opto_times = [100 5]
# parse_opto_times(opto_times, mp)


3×2 Array{Any,2}:
 2.71828   3
 6         8
 9        10

# Defining default model params, and make_input() and run_ntrials()

In [95]:
#@include_me  pro_anti.jl

model_params = Dict(
:dt     =>  0.02,    # timestep, in secs
:tau    =>  0.1,     # tau, in ms
:vW     =>  -1.7,    # vertical weight
:hW     =>  -1.7,    # horizontal weight
:sW     =>  0.2,     # self-connection weight
:dW     =>  0,       # diagonal weight
:nsteps =>  2,       # number of timesteps in the simulation
:noise  =>  [],      # noise added during simulation. Can be empty matrix, or an nunits-by-nsteps matrix
:sigma  =>  0.08,    # standard deviation of Gaussian noise added (will be scaled by sqrt(dt) to be relatively dt-INsensitive)
:input  =>  0,       # input current. Can be scalar, nunits-by-1, or nunits-by-nsteps matrix
:g_leak =>  0.25,    # leak conductance
:U_rest =>  -1,      # resting membrane potential
:theta  =>  1,       # inverse slope of g() function
:beta   =>  1,       # offset to g() function
:constant_excitation      => 0.19,   # constant input, added to all units at all timesteps
:anti_rule_strength       => 0.1,    # input added only to anti units during rule_and_delay_period in Anti trials
:pro_rule_strength        => 0.1,    # input added only to pro units during rule_and_delay_period in Pro trials
:const_pro_bias           => 0,      # input added only to pro units during all times in all trial types
:target_period_excitation => 0.2,      # input added to all units during target_period
:right_light_excitation   => 0.3,    # input added to the Anti and the Pro unit on one side during the target_period
:right_light_pro_extra    => 0,      # input added to the right side Pro unit alone during the target_period
:rule_and_delay_period    => 0.4,    # duration of rule_and_delay_period, in secs
:target_period            => 0.1,    # duration of target_period, in secs
:post_target_period       => 0.5,    # duration of post_target_period, in secs
:const_add => 0,  # from rate_networks.jl, unused here
:init_add  => 0,  # from rate_networks.jl, unused here 
:opto_strength            => 1,      # fraction by which to scale V outputs
:opto_times               => [],     # n-by-2 matrix, indicating start and stop times for opto_strength effect, all within the same trial
:opto_units               => 1:4,    # ids of the units that will be affected by opto_strength effect
)


"""
input, t, nsteps = make_input(trial_type; dt=0.02, nderivs=0, difforder=0, constant_excitation=0.19, anti_rule_strength=0.1, 
    pro_rule_strength=0.1, target_period_excitation=1, right_light_excitation=0.5, right_light_pro_extra=0, 
    rule_and_delay_period=0.4, target_period=0.1, post_target_period=0.4, const_pro_bias=0,
    other_unused_params...)
"""
function make_input(trial_type; dt=0.02, nderivs=0, difforder=0, constant_excitation=0.19, anti_rule_strength=0.1, 
    pro_rule_strength=0.1, target_period_excitation=1, right_light_excitation=0.5, right_light_pro_extra=0, 
    rule_and_delay_period=0.4, target_period=0.1, post_target_period=0.4, const_pro_bias=0,
    other_unused_params...)

    if FDversion() >= 0.6
        # All the variables that we MIGHT choose to differentiate w.r.t. go into this bag -- further down
        # we'll use get_eltype(varbag) to check for any of them being ForwardDiff.Dual.
        # That is how we'll tell whether new matrices should be regular numbers of ForwardDiff.Dual's.
        # *** if you add a new variable you'll want to differentiate w.r.t., it should be added here too ***
        varbag = (dt, constant_excitation, anti_rule_strength, pro_rule_strength, target_period_excitation, 
        right_light_excitation, right_light_pro_extra, rule_and_delay_period, target_period, post_target_period,
        const_pro_bias)
    end
    
    T = rule_and_delay_period + target_period + post_target_period
    t = 0:dt:T
    nsteps = length(t)
    
    if FDversion() < 0.6
        input = constant_excitation + ForwardDiffZeros(4, nsteps, nderivs=nderivs, difforder=difforder)
    else
        input = constant_excitation + zeros(get_eltype(varbag), 4, nsteps)
    end
    
    if trial_type=="Anti"
        input[2:3, t.<rule_and_delay_period] += anti_rule_strength
    elseif trial_type=="Pro"
        input[[1,4], t.<rule_and_delay_period] += pro_rule_strength
    else
        error("make_input: I don't recognize input type \"" * trial_type * "\"")
    end
    
    if VERSION.major + 0.1*VERSION.minor < 0.6
        input[:,     (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += target_period_excitation
        input[1:2,   (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += right_light_excitation
        input[1,     (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += right_light_pro_extra
    else    
        input[:,     (rule_and_delay_period.<=t) .& (t.<rule_and_delay_period+target_period)] += target_period_excitation
        input[1:2,   (rule_and_delay_period.<=t) .& (t.<rule_and_delay_period+target_period)] += right_light_excitation
        input[1,     (rule_and_delay_period.<=t) .& (t.<rule_and_delay_period+target_period)] += right_light_pro_extra
    end    
    input[[1,4],:] += const_pro_bias
    
    return input, t, nsteps
end



"""
proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = 
    run_ntrials(nPro, nAnti; plot_list=[], start_pro=[-0.5,-0.5,-0.5,-0.5], start_anti=[-0.5,-0.5,-0.5,-0.5],
        profig=1, antifig=2, plot_Us=true, clearfig=true, ax_set=Dict(), 
        opto_units = 1:4, nderivs=0, difforder=0, model_params...)

Runs a set of proAnti model trials.  See model_params above for definition of all the parameters. In addition,

# PARAMETERS

- nPro    number of Pro tials to run

- nAnti   number of Anti trials to run

# OPTIONAL PARAMETERS

- plot_list    A list of trials to plot on the figures. If empty nothing is plotted

- profig       The figure number on which Pro traces will be drawn

- antifig      The figure number on which Anti traces will be drawn

- start_pro    A 4-by-1 vector of starting U values on Pro trials for the 4 units

- start_anti   A 4-by-1 vector of starting U values on Anti trials for the 4 units

- nderivs      For ForwardDiff

- difforder    For ForwardDiff

- clearfig     if ax_set is not empty, then clearfig=true clears the figure

- ax_set       If passed, should be a vector with two elements, the first for the Pro
               axes, the second for the Anti axes. Each element should itself be a vector,
               composed of axis handles that will be used for plotting, of length equal to
               plottables.

               Alternatively, for backwards compatibility, ax_set may be a Dict(), 
               with keys "pro_Vax", "pro_Uax", "pro_Dax" and "anti_Vax", "anti_Uax", "anti_Dax" 
               whose values are corresponding axis handles, to be passed to `plot_PA()`

- plot_Us      For backwards compatibility (for updated usage, see `plottables` in
               OPTIONAL PARAMS below.) If passed, should be either true (plot also the Us) or false
               (drop the Us). The new `plottables` is much more flexible.

- model_params   Further optional params, will be passed onto forwardModel() (e.g., opto times, although
                opto_times is first passed through parse_opto_times())

# FURTHER OPTIONAL PARAMETERS THAT WILL BE PASSED ON TO PLOT_PA()

- plottables   A vector of strings. Each of these strings indicates something to 
            plot; the strings will be evaluated in a context where the variables "V", "U", 
            and "t" are instantiated to have the values passed to `plot_PA()`. Thus 
            plottables=["V", "V[1,:]-V[4,:]"] indicates that two axes should be used, on
            the first one V will be plotted, on the second V[1,:]-V[4,:].
                The strings can be any arbitrary Julia expression, with the condition that
            their evaluation should produce something with either nrows or ncolumns
            equal to length(t).

- ylabels   A vector of strings, should be equal in length to plottables; Each element
            here will be used as the y label for the corresponding axis.

- ylims     A vector of Any, should be in length to plottables. If an element is
            `nothing`, then allow automatic scaling of the y axes for this axis. Otherwise,
            the element should be a 2-long vector of Float64, indicating the minimum and
            the maximum for the y axis, respectively.

- ax_set    If passed, should be a vector of axis handles, equal in length to 
            plottables. If not passed, new axes will be created, vertically stacked, number 
            equal to length of plottables.

            Finally, for backwards compatibility, ax_set may also be a Dict, in which case it is 
            expected to have keys "Vax", "Uax", and "Dax", each with value equal to an axis handle, 
            on which V, U, and V[1,:] - V[4,:] will be plotted. In this backwards compatibility
            mode, if plot_Us=false, then U is not plotted and "Uax" is not needed.

- plot_Us  If passed, should be either true or false, and indicates that we are in 
            backwards compatibility mode (see ax_set above).

- singleton_color   If the result of evaluating plottables[i] has one dimension
            of length 1 (i.e., it is a vector, not a matrix) then that line will be plotted in
            this color.

- color_list If all dimensions of the result of evaluating plottables[i] are bigger than 1,
            then the color order of the resulting lines plotted will be the rows of color_list

- fignum   Which figure to plot on; only used if ax_set is empty

- clearfig   If ax_set is empty and clearfig=true, clears the figure before doing anything



# RETURNS

- proVs    final V for the four units on Pro trials

- antiVs   final V for the four units on Anti trials

- pro_fullVs   all Vs, across all times, for all Pro trials (4-by-nsteps-by-nPro)

- anti_fullVs  all Vs, across all times, for all Anti trials (4-by-nsteps-by-nAnti)


# EXAMPLE
```jldoctest
proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
    run_ntrials(nPro, nAnti; plot_list=[1:5;], 
    plottables = ["V", "(V[1,:]+V[4,:]) - (V[2,:]+V[3,:])", "V[1,:]-V[4,:]"],
    ylabels = ["V", "Pro - Anti rule encoding", "Pro_R - Pro_L"],
    ylims = [[-0.02, 1.02], nothing, nothing],
    my_params...)
```
"""
function run_ntrials(nPro, nAnti; plot_list=[], start_pro=[-0.5,-0.5,-0.5,-0.5], start_anti=[-0.5,-0.5,-0.5,-0.5],
    profig=1, antifig=2, clearfig=true, ax_set = nothing,
    opto_units = 1:4, nderivs=0, difforder=0, model_params...)

    if FDversion() >= 0.6
        # All the variables that we MIGHT choose to differentiate w.r.t. go into this bag -- further down
        # we'll use get_eltype(varbag) to check for any of them being ForwardDiff.Dual.
        # That is how we'll tell whether new matrices should be regular numbers of ForwardDiff.Dual's.
        # *** if you add a new variable you'll want to differentiate w.r.t., it should be added here too ***
        varbag = (start_pro, start_anti, model_params)
        # print("get_eltype(varbag)="); print(get_eltype(varbag)); print("\n")
    end
    
    pro_ax_set  = nothing
    anti_ax_set = nothing
    if typeof(ax_set)<:Dict
        pro_ax_set  = Dict()
        anti_ax_set = Dict()
        if haskey(ax_set, "pro_Vax");  pro_ax_set["Vax"]  = ax_set["pro_Vax"]; end;
        if haskey(ax_set, "pro_Uax");  pro_ax_set["Uax"]  = ax_set["pro_Uax"]; end;
        if haskey(ax_set, "pro_Dax");  pro_ax_set["Dax"]  = ax_set["pro_Dax"]; end;                        
        if haskey(ax_set, "anti_Vax"); anti_ax_set["Vax"] = ax_set["anti_Vax"]; end;
        if haskey(ax_set, "anti_Dax"); anti_ax_set["Dax"] = ax_set["anti_Dax"]; end;
        if haskey(ax_set, "anti_Uax"); anti_ax_set["Uax"] = ax_set["anti_Uax"]; end;

        # If we were passed pro and anti axes, their figure numbers override profig and antifig:
        if haskey(ax_set, "pro_Vax");  profig =ax_set["pro_Vax"][:figure][:number];  end
        if haskey(ax_set, "anti_Vax"); antifig=ax_set["anti_Vax"][:figure][:number]; end
    elseif ax_set != nothing
        pro_ax_set  = ax_set[1]
        anti_ax_set = ax_set[2]

        # If we were passed pro and anti axes, their figure numbers override profig and antifig:
        profig  = pro_ax_set[1][:figure][:number]
        antifig = anti_ax_set[1][:figure][:number]
    end

    model_params = Dict(model_params)
    pro_input,  t, nsteps = make_input("Pro" ; nderivs=nderivs, difforder=difforder, model_params...)
    anti_input, t, nsteps = make_input("Anti"; nderivs=nderivs, difforder=difforder, model_params...)
    
    sW = model_params[:sW]
    hW = model_params[:hW]
    vW = model_params[:vW]
    dW = model_params[:dW]
    model_params = make_dict(["nsteps", "W"], [nsteps, [sW vW dW hW; vW sW hW dW; dW hW sW vW; hW dW vW sW]], 
        model_params)
    model_params = make_dict(["nderivs", "difforder"], [nderivs, difforder], model_params)
    model_params[:opto_times] = parse_opto_times(model_params[:opto_times], model_params)
    
    if FDversion()<0.6
        proVs  = ForwardDiffZeros(4, nPro, nderivs=nderivs, difforder=difforder)
        antiVs = ForwardDiffZeros(4, nAnti, nderivs=nderivs, difforder=difforder)
    else
        proVs  = zeros(get_eltype(varbag), 4, nPro)
        antiVs = zeros(get_eltype(varbag), 4, nAnti)
    end
    proVall  = zeros(4, nsteps, nPro);
    antiVall = zeros(4, nsteps, nAnti);
    # --- PRO ---
    if length(plot_list)>0; figure(profig); if (pro_ax_set==nothing) && clearfig; clf(); end; end
    model_params = make_dict(["input"], [pro_input], model_params)
    for i=1:nPro
        Uend, Vend, pro_fullU, temp = forwardModel(start_pro, do_plot=false, opto_units=opto_units; model_params...)
        if FDversion() < 0.6; proVall[:,:,i] = temp; else proVall[:,:,i] = get_value(temp); end
        # print("typeof(proVs) = "); print(typeof(proVs)); print("\n")
        proVs[:,i] = Vend
        if any(plot_list.==i) 
            plot_PA(t, get_value(pro_fullU), get_value(proVall[:,:,i]); clearfig=false,
                fignum=profig, ax_set=pro_ax_set, model_params...)
        end
    end

    # --- ANTI ---
    if length(plot_list)>0; figure(antifig); if (anti_ax_set==nothing) && clearfig; clf(); end; end
    model_params = make_dict(["input"], [anti_input], model_params)
    for i=1:nAnti
        startU = [-0.5, -0.5, -0.5, -0.5]
        Uend, Vend, anti_fullU, temp = forwardModel(start_anti, do_plot=false, opto_units=opto_units; model_params...)
        if FDversion() < 0.6; antiVall[:,:,i] = temp; else antiVall[:,:,i] = get_value(temp); end
        antiVs[:,i] = Vend
        if any(plot_list.==i) 
            plot_PA(t, get_value(anti_fullU), get_value(antiVall[:,:,i]); clearfig=false,
            fignum=antifig, ax_set=anti_ax_set, model_params...)
        end
    end
        
    if haskey(model_params, :opto_fraction)
        opto_fraction = model_params[:opto_fraction]
    else
        opto_fraction = 1
    end
    
    return proVs, antiVs, proVall, antiVall, opto_fraction, pro_input, anti_input
end

run_ntrials

## example of running run_ntrials() with opto

In [85]:
pygui(true)
nPro = 10; nAnti = 5;

# middle fifth of target period:
opto_times = ["target_start + 0.4*(target_end-target_start)"  "target_start + 0.6*(target_end-target_start)"]

# middle third of rule and delay
opto_times = ["0.333*target_start"  "0.666*target_start"]

my_params = merge(model_params, Dict(:opto_strength=>0.01, :opto_times=>opto_times))

proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
    @time(run_ntrials(nPro, nAnti; plot_list=[1:5;], my_params...))

@printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/nPro)
@printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/nAnti)

  1.492171 seconds (471.34 k allocations: 20.441 MiB, 0.69% gc time)


QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap


Pro % correct = 80%
Anti % correct = 40% 


#### example of run_ntrials() with axis placing

In [86]:
# Another example, demonstrating axes placing

pygui(true)
nPro = 10; nAnti = 10;

figure(1); clf();


# middle third of rule and delay
opto_times = ["0.333*target_start"  "0.666*target_start"]
my_params = merge(model_params, Dict(:opto_strength=>0.01, :opto_times=>opto_times))

n=1
proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
run_ntrials(nPro, nAnti; plot_list=[1:5;], profig=1, antifig=1, ax_set=Dict("pro_Vax"=>subplot(4,3,n),
"pro_Dax" => subplot(4,3,n+3), "anti_Vax"=>subplot(4,3,n+6), "anti_Dax"=>subplot(4,3,n+9)), plot_Us=false, my_params...)


# target period
opto_times = ["target_start"  "target_end"]
my_params = merge(model_params, Dict(:opto_strength=>0.01, :opto_times=>opto_times))

n=2
proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
run_ntrials(nPro, nAnti; plot_list=[1:5;], profig=1, antifig=1, ax_set=Dict("pro_Vax"=>subplot(4,3,n),
"pro_Dax" => subplot(4,3,n+3), "anti_Vax"=>subplot(4,3,n+6), "anti_Dax"=>subplot(4,3,n+9)), plot_Us=false, my_params...)


# post_target period
opto_times = ["target_end", "trial_end"]
my_params = merge(model_params, Dict(:opto_strength=>0.01, :opto_times=>opto_times))

n=3
proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
run_ntrials(nPro, nAnti; plot_list=[1:5;], profig=1, antifig=1, ax_set=Dict("pro_Vax"=>subplot(4,3,n),
"pro_Dax" => subplot(4,3,n+3), "anti_Vax"=>subplot(4,3,n+6), "anti_Dax"=>subplot(4,3,n+9)), plot_Us=false, my_params...);




#### example of run_ntrials() with plottables expression evaluation

In [96]:
pygui(true)
nPro = 10; nAnti = 5;

# middle fifth of target period:
opto_times = ["target_start + 0.4*(target_end-target_start)"  "target_start + 0.6*(target_end-target_start)"]

# middle third of rule and delay
opto_times = ["0.333*target_start"  "0.666*target_start"]

my_params = merge(model_params, Dict(:opto_strength=>0.01, :opto_times=>opto_times))

proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
    run_ntrials(nPro, nAnti; plot_list=[1:5;], 
    plottables = ["V", "(V[1,:]+V[4,:]) - (V[2,:]+V[3,:])", "V[1,:]-V[4,:]"],
    ylabels = ["V", "Pro - Anti rule encoding", "Pro_R - Pro_L"],
    ylims = [[-0.02, 1.02], nothing, nothing],
    my_params...)

@printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/nPro)
@printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/nAnti)

Pro % correct = 80%
Anti % correct = 40% 


In [93]:
V = randn(4,10)
replacer("sqrt.(V[1,:])", Dict("V"=>[1,2,3]))

LoadError: [91msyntax: malformed expression[39m

### example differentiation of run_ntrials

In [11]:

func = (;pars...) -> begin 
    fullpars = merge(my_params, Dict(pars))
    proVs, antiVs, pro_fullV, anti_fullV, opto_strength, pro_input, anti_input = 
        run_ntrials(nPro, nAnti; plot_list=[1:5;], fullpars...)
    return(sum(proVs))
end

args = ["vW", "hW"]
pars = [-1.7, -1.8]
# First time, with JIT compilation of ForwardDiff for this func, is slow
value, grad, hess = @time(keyword_vgh(func, args, pars))

# Second time is faster
value, grad, hess = @time(keyword_vgh(func, args, pars))



  3.809880 seconds (1.93 M allocations: 102.487 MiB, 1.38% gc time)
  0.922851 seconds (139.93 k allocations: 14.340 MiB, 0.79% gc time)


(0.029031191994365386, [0.00132497, 0.00129442], [0.000313657 0.000314931; 0.000314931 0.000318346])

# Defining JJ(), the cost function

Below is a cost function `JJ()` with two terms. Let $C$ be the target fraction "Go Right" trials, and $V_R$ and $V_L$ represent the final value of the $V$ variable for the "Pro Go Right" and "Pro Go Left" units, respectively.

$$
cost_1 = \left( \left\langle \frac{1}{2} + \frac{1}{2}\tanh(\frac{V_R - V_L}{\theta_1}) \right\rangle_{trials} - C \right)^2
$$

In the limit of $\theta_1$ very small, the network's ouput is binarized, decisions go like the sign of $V_R-V_L$, and this cost function is minimized at the desired fraction correct. However, the perfectly binary network is not differentiable. So we keep $\theta1$ smal but non-zero.

To try to make sure that output decisions are clear, we have another component to the cost function

$$
cost_2 = - \left\langle \left( \tanh(\frac{V_R - V_L}{\theta_2}) \right)^2 \right \rangle_{trials}
$$

This component is minimized when the difference between $V_R$ and $V_L$ is large, but if differences are much bigger than $\theta_2$, it stops caring about them.

We've been using a cost function that is a combination of the two:

$$
J_{\rm total} = cost_1 + \beta \cdot cost_2
$$

Meaning our total cost function has three parameters, $\theta_1$, $\theta_2$, and $\beta$, which is a bit of a mess.


In [12]:
figure(1); clf();
th1 = 0.05
th2 = 0.25
b   = 0.01
C   = 0.7

p   = 1

vrmvl = -1:0.01:1
cost1 = (0.5 + 0.5*tanh.(vrmvl/th1) - C).^2
cost2 = -tanh.((vrmvl/th2).^p).^2

ax1 = subplot(3,1,1); plot(vrmvl, cost1)
remove_xtick_labels(ax1)
ylabel("cost_1")
title("single-trial cost_1, shown here, not so meaningful")

ax2 = subplot(3,1,2); plot(vrmvl, cost2)
remove_xtick_labels(ax2)
ylabel("cost_2")

ax3 = subplot(3,1,3); plot(vrmvl, cost1 + b*cost2)
xlabel("V_R - V_L")
ylabel("cost_1 + beta*cost_2")



PyObject Text(42.8472,0.5,'cost_1 + beta*cost_2')

## Example of loading a farm and running JJ() on it.

In [ ]:
pygui(true); hP = 0; hA = 0

F = matread("FarmFields/farm_LD0003.mat")
model_params = symbol_key_ize(F["model_params"])
model_params[:rule_and_delay_periods] = F["rule_and_delay_periods"]
model_params[:post_target_periods] = F["post_target_periods"]
model_params[:seedrand]=F["test_sr"]
model_params[:cbeta] =F["cb"]
model_params[:start_pro] = [-0.5, -0.5, -0.5, -0.5]
model_params[:start_anti] = [-0.5, -0.5, -0.5, -0.5]
model_params = make_dict(F["args"], F["pars"], model_params)

# Julia 0.5.2 takes 21 secs for next line
# Julia 0.6 takes 10.4 secs for next line
cost, cost1s, cost2s, hP,hA,dP,dA,hBP,hBA = @time(JJ(100, 100; 
plot_list=[1:10;], plot_conditions=[false,false,false,false, false], verbose=true, model_params...));

# Julia 0.5.2 takes 9.3 secs for next line
# Julia 0.6 takes 8.18 secs for next line
cost, cost1s, cost2s, hP,hA,dP,dA,hBP,hBA = @time(JJ(100, 100; 
plot_list=[1:10;], plot_conditions=[false,false,false,false, false], verbose=true, model_params...));

[["control", "full", "rule", "delay", "target"] hP hA]


In [73]:
#@include_me  pro_anti.jl

"""
cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, [proValls, antiValls, opto_fraction, pro_input, anti_input] = 
    JJ(nPro, nAnti; pro_target=0.9, anti_target=0.7, 
        opto_targets = Array{Float64}(0,2), opto_periods = Array{Float64}(0,2), 
        model_details = false,
        theta1=0.025, theta2=0.035, cbeta=0.003, verbose=false, verbose_file = STDOUT,
        pre_string="", zero_last_sigmas=0, seedrand=NaN, 
        rule_and_delay_periods = nothing, target_periods = nothing, post_target_periods = nothing,
        plot_conditions=false, plot_list = [],
        nderivs=0, difforder=0, model_params...)

Runs a proAnti network, if desired across multiple opto conditions and multiple period durations and returns
resulting cost.

If rule_and_delay_periods and post_target_periods are not passed, it tries to get them from their
singular (not plural) versions in model_params, e.g., model_params[:rule_and_delay_period]. NOTE that
target_period is different, for backwards compatibility it defaults to 0.1.

# PARAMETERS (INCOMPLETE DOCS!!!):

- nPro, nAnti    The number of Pro and the number of Anti trials to run

- pro_target     The target %correct for Pro trials

- anti_target    The target %correct for Anti trials

- opto_targets   Each row is a [target % correct Pro, %correct Anti] for the corresponding row of opto_periods

- opto_periods  Each row is the [start_time, end_time] for an opto condition. These follow the rules
                of `parse_opto_times()`: they can be arbitrary Julia expressions involving the terms 
                trial_start, target_start, target_end, and trial_end.

- model_details   If this is set to true, the function will also returns proValls, antiValls, 
                opto_fraction, pro_input, anti_input (see below)

- theta1        Trials are interpreted as going either Left or Right based on the sign of V_pro_L - V_pro_R.
                However, to make this a continuous function, went_Left is actually computed as the continuous 
                function 0.5(1 + tanh((V__pro_L - V_pro_R)/theta1)). In other words, theta1 is the width
                of the smoothing band.

- theta2        To encourage outputs to lie *outside* the smoothing band, we also compute
                -tanh((V_pro_L - V_pro_R)/theta2)^2, and make that part of the cost. This encourage differences
                between V_pro_L and V_pro_R that are larger than theta2, but cares little if the differences are 
                already larger than theta2.  This cost function term is added after being multiplied by factor cbeta

- cbeta         The cost function is going to be (theta1 term) + cbeta(theta2 term).  We typically keep
                cbeta small, so that at first only the theta1 term matters; once that theta1 term starts 
                getting close to its target value (and so its cost gets small, approaching zero) then the 
                theta2 term becomes, relatively speaking, important.

- seedrand      If sets, calls srand() on the value to initialize the random number generator.

- verbose       If true, prints out diagnostic information to the console.

- verbose_file  If other than STDOUT, should be a string indicating a filename that verbose
                output will be written to.

- pre_string    Relevant only under verbose=true, a string that gets printed out before the rest of the verbose info.

- rule_and_delay_periods    Vector, indicating set of rule_and_delay_period lengths to iterate over, 
                            while testing set of opto_periods, etc. on each one. 
                            Deafult is to do a single one, as picked out from model_params[:rule_and_delay_period]

- target_periods            Vector, indicating set of target_perdiod lengths to iterate over, 
                            while testing set of opto_periods, etc. on each one.
                            ** DEFAULT IS TO USE 0.1**, not to pick it out from model_params

- post_target_periods       Vector, indicating set of post_target_period lengths to iterate over, 
                            while testing set of opto_periods, etc. on each one. 
                            Deafult is to do a single one, as picked out from model_params[:post_target_period]
 

- plot_conditions=false,    If anything other than false, is expected to be a list of Booleans, same
                            length as the number of rows of opto_periods; ture indicates that the 
                            corresponding opto_period should be plotted.

- plot_list                 A list of trial numbers to plot in each condition

- model_params              Any remaining keyword-value params are passed as is on to run_ntrials 


# RETURNS (INCOMPLETE DOCS!!!):

- cost   The net cost, composed of squared error cost (promoting performance close to the desired one) plus difference cost (promoting Pro_R and Pro_L differences at the end of the trial)

- cost1s  A vector, for each opto condition, the squared error cost

- cost2s  for each opto condition, the differences costc

- hP     Pro "hits", as computed with the theta1 sigmoid
    
- hA     Anti "hits"

- dP     Pro "diffs", as computed with the theta2 sigmoid
    
- dA     Anti "diffs"

- hBP    Pro binarized hits, as computed by binarizing (equivalent to theta1->0)
    
- hBA    Anti binarized hits

If model_details is set to true, also returns the following (nopto is number of opto conditions, nruns_each is total number of unique period duration combinations)

- proValls          Array, nopto-by-nruns_each of Float64 Arrays, each of which is 4-by-nsteps-nPro and is V versus time for each trial

- antiValls         Array, nopto-by-nruns_each of Float64 Arrays, each of which is 4-by-nsteps-nPro and is V versus time for each trial

- opto_fraction

- pro_input

- anti_input



"""

function JJ(nPro, nAnti; pro_target=0.9, anti_target=0.7, 
    opto_targets = Array{Float64}(0,2), opto_periods = Array{Float64}(0,2), 
    model_details = false,
    theta1=0.025, theta2=0.035, cbeta=0.003, verbose=false, verbose_file = STDOUT,
    pre_string="", zero_last_sigmas=0, seedrand=NaN, 
    rule_and_delay_periods = nothing, target_periods = nothing, post_target_periods = nothing,
    plot_conditions=false, plot_list = [],
    nderivs=0, difforder=0, model_params...)

    if FDversion() >= 0.6
        # All the variables that we MIGHT choose to differentiate w.r.t. go into this bag -- further down
        # we'll use get_eltype(varbag) to check for any of them being ForwardDiff.Dual.
        # That is how we'll tell whether new matrices should be regular numbers of ForwardDiff.Dual's.
        # *** if you add a new variable you'll want to differentiate w.r.t., it should be added here too ***
        varbag = (pro_target, anti_target, opto_targets, opto_periods, theta1, theta2, cbeta,model_params)
        # print("get_eltype(varbag)="); print(get_eltype(varbag)); print("\n")
    end
    
    # If the plurals of the periods are not passed in, then use the singular in model_params as the default:
    if rule_and_delay_periods==nothing
        rule_and_delay_periods = model_params[:rule_and_delay_period]
    end
    if target_periods==nothing        
        target_periods = [0.1]
        if verbose_file==STDOUT; ostr=STDOUT else ostr=open(verbose_file, "a"); end
        @printf(ostr, "\n\nWARNING: JJ() was not given a target_periods parameter, I will *IGNORE* the\n")
        @printf(ostr, "    model_params[:target_period] entry and will use a default of 0.1\n\n")
        if verbose_file!=STDOUT; close(ostr); end
    end
    if post_target_periods==nothing
         post_target_periods = model_params[:post_target_period]
    end
    
    if size(opto_targets,1)==0  || size(opto_periods,1)==0 # if there's no opto that is being asked for
        # Then tun with only a single opto_period request, with no opto, and control targets as our targets
        opto_targets = [pro_target anti_target]
        opto_periods = [-1, -1]   # opto effect is before time zero, i.e., is nothing
    end
    
    if size(opto_periods,2) != 2
        try
            opto_periods = reshape(opto_periods, Int64(round(length(opto_periods)/2)), 2)  # Make sure its rows of 2 cols
        catch
            error("Something is wrong with opto_periods -- it should have 2 columns")
        end
    end
    if size(opto_targets,2) != 2
        try
            opto_targets = reshape(opto_targets, Int64(round(length(opto_targets)/2)), 2)  # Make sure its rows of 2 cols
        catch
            error("Something is wrong with opto_targets -- it should have 2 columns")
        end
    end
    
    if ~(size(opto_targets) == size(opto_periods)); 
        error("opto parameters are bad -- need a Pro and Anti performance target for each requested period"); 
    end

    noptos     = size(opto_periods,1)  # of opto conditions
    nruns_each = length(rule_and_delay_periods)*length(target_periods)*length(post_target_periods)    # runs per opto condition
    nruns      = nruns_each*noptos  # total conditions

    # --- WARNING!!!! BUG HERE, Should have been nruns_each, not nruns. Keeping 
    # it as is for backwards compatibility, but all final costs will come out divided
    # by noptos (because there will be that factor of columns of untouched zeros)
    if FDversion() < 0.6
        cost1s = ForwardDiffZeros(noptos, nruns, nderivs=nderivs, difforder=difforder)
        cost2s = ForwardDiffZeros(noptos, nruns, nderivs=nderivs, difforder=difforder)
    else
        cost1s = zeros(get_eltype(varbag), noptos, nruns)
        cost2s = zeros(get_eltype(varbag), noptos, nruns)
    end
            
    hP  = zeros(noptos, nruns_each);   # Pro "hits", as computed with the theta1 sigmoid
    hA  = zeros(noptos, nruns_each);   # Anti "hits"
    dP  = zeros(noptos, nruns_each);   # Pro "diffs", as computed with the theta2 sigmoid
    dA  = zeros(noptos, nruns_each);   # Anti "diffs"
    hBP = zeros(noptos, nruns_each);   # Pro binarized hits, as computed by binarizing (equivalent to theta1->0)
    hBA = zeros(noptos, nruns_each);   # Anti binarized hits

    proValls         = Array{Array{Float64}}(noptos, nruns_each);
    antiValls        = Array{Array{Float64}}(noptos, nruns_each);
    opto_fraction    = [];
    pro_input        = [];
    anti_input       = [];
    
    for nopto=1:noptos # iterate over each opto inactivation period
        # @printf(ostr, "size(hBP) is %d, %d\n", size(hBP,1), size(hBP,2))

        # reset random number generator for each opto period, so it cant over fit noise samples
        if ~isnan(seedrand); srand(seedrand); end

        n = 0  # n is a counter over all period duration conditions
        totHitsP = totHitsA = totDiffsP = totDiffsA = 0
        for i in rule_and_delay_periods
            for j in target_periods
                for k = post_target_periods
                    n += 1

                    # include this opto inactivation in the parameters to pass on
                    my_params = make_dict(["rule_and_delay_period","target_period","post_target_period"], [i,j,k])
                    my_params = make_dict(["opto_times"], [reshape(opto_periods[nopto,:], 1, 2)], my_params)
                    my_params = merge(Dict(model_params), my_params)  # my_params takes precedence

                    my_plot_list = [];
                    if typeof(plot_conditions)==Bool && plot_conditions
                        my_plot_list = plot_list;
                    elseif typeof(plot_conditions)<:Array && plot_conditions[nopto]
                        my_plot_list = plot_list;
                    end

                    # print("model params is " ); print(model_params); print("\n")
                    proVs, antiVs, proVall, antiVall, opto_fraction,pro_input,anti_input =
                        run_ntrials(nPro, nAnti; plot_list=my_plot_list, 
                            nderivs=nderivs, difforder=difforder, my_params...)
                        # run_ntrials_opto(nPro, nAnti; nderivs=nderivs, difforder=difforder, my_params...)

                    # if length(proValls)==0
                    #    proValls = zeros(4, size(proVall,2), size(proVall,3), noptos)
                    # end
                    # if length(antiValls)==0
                    #    antiValls = zeros(4, size(antiVall,2), size(antiVall,3), noptos)
                    # end
                    # proValls[:,:,:,nopto]  = get_value(proVall)
                    # antiValls[:,:,:,nopto] = get_value(antiVall)
                    # @printf(ostr, "size of proValls is "); print(size(proValls)); print("\n")
                    # print("size(proValls)="); print(size(proValls)); print("\n")
                    proValls[nopto,  n] = proVall
                    antiValls[nopto, n] = antiVall
                    
                    hitsP  = 0.5*(1 + tanh.((proVs[1,:]-proVs[4,:,])/theta1))
                    diffsP = tanh.((proVs[1,:,]-proVs[4,:])/theta2).^2
                    hitsA  = 0.5*(1 + tanh.((antiVs[4,:]-antiVs[1,:,])/theta1))
                    diffsA = tanh.((antiVs[4,:,]-antiVs[1,:])/theta2).^2

                    # set up storage  -- we do get_value() to make sure to from ForwardDiff.Dual into Float64 if necessary
                    hP[nopto, n] = mean(get_value(hitsP));
                    hA[nopto, n] = mean(get_value(hitsA));
                    dP[nopto, n] = mean(get_value(diffsP));
                    dA[nopto, n] = mean(get_value(diffsA));
                    hBP[nopto, n] = get_value(sum(proVs[1,:] .>= proVs[4,:,])/nPro);
                    hBA[nopto, n] = get_value(sum(antiVs[4,:] .>  antiVs[1,:,])/nAnti);                    
                    
                    if nPro>0 && nAnti>0
                        # cost1s and cost2s can accept ForwardDiff.Dual, so no get_value() for them
                        cost1s[nopto, n] = (nPro*(mean(hitsP) - opto_targets[nopto,1]).^2  
                                    + nAnti*(mean(hitsA) - opto_targets[nopto,2]).^2)/(nPro+nAnti)
                        cost2s[nopto, n] = -cbeta*(nPro*mean(diffsP) + nAnti*mean(diffsA))/(nPro+nAnti)
                    elseif nPro>0
                        cost1s[nopto, n] = (mean(hitsP) - opto_targets[nopto,1]).^2
                        cost2s[nopto, n] = -cbeta*mean(diffsP)
                    else
                        cost1s[nopto, n] = (mean(hitsA) - opto_targets[nopto,2]).^2
                        cost2s[nopto, n] = -cbeta*mean(diffsA)
                    end
                    totHitsP  += mean(hitsP);  totHitsA  += mean(hitsA); 
                    totDiffsP += mean(diffsP); totDiffsA += mean(diffsA);
                end
            end
        end
        ## @printf(ostr, "size(hBP) is %d, %d\n", size(hBP,1), size(hBP,2))
        hitsP = totHitsP/n; hitsA = totHitsA/n; diffsP = totDiffsP/n; diffsA = totDiffsA/n
    
        if verbose
            if verbose_file==STDOUT; ostr=STDOUT else ostr=open(verbose_file, "a"); end            
            pcost1 = mean(cost1s[nopto,:])   # partial costs
            pcost2 = mean(cost2s[nopto,:])            
            
            # Notice the get_value() calls below, to transform ForwardDiff Duals into Float64s
            @printf(ostr, "%s", pre_string)
            @printf(ostr, "Opto condition # %d\n", nopto)
            @printf(ostr, "     - %d - cost=%g, cost1=%g, cost2=%g\n", nopto,
                get_value(pcost1+pcost2), get_value(pcost1), get_value(pcost2))
            if nPro>0 && nAnti>0
                @printf(ostr, "     - %d - mean(hitsP)=%g, mean(diffsP)=%g mean(hitsA)=%g, mean(diffsA)=%g\n", nopto,
                    get_value(mean(hitsP)), get_value(mean(diffsP)),
                    get_value(mean(hitsA)), get_value(mean(diffsA)))
            elseif nPro>0
                @printf(ostr, "     - %d - mean(hitsP)=%g, mean(diffsP)=%g (nAnti=0)\n", nopto,
                    get_value(mean(hitsP)), get_value(mean(diffsP)))
            else
                @printf(ostr, "     - %d - (nPro=0) mean(hitsA)=%g, mean(diffsA)=%g\n", nopto,
                    get_value(mean(hitsA)), get_value(mean(diffsA)))
            end        
            if verbose_file!=STDOUT; close(ostr); end
        end
    end
        
    # @printf(ostr, "size(hBP) is %d, %d\n", size(hBP,1), size(hBP,2))

    cost1 = mean(cost1s)
    cost2 = mean(cost2s)

    if verbose
        if verbose_file==STDOUT; ostr=STDOUT else ostr=open(verbose_file, "a"); end
        @printf(ostr, "%s", pre_string)
        @printf(ostr, "OVERALL\n")
        @printf(ostr, "     -- cost=%g, cost1=%g, cost2=%g\n", 
            get_value(cost1+cost2), get_value(cost1), get_value(cost2))
        if verbose_file!=STDOUT; close(ostr); end
    end
    
    if model_details
        return cost1 + cost2, cost1s, cost2s, hP,hA,dP,dA,hBP,hBA, 
            proValls, antiValls, opto_fraction, pro_input, anti_input
    else
        return cost1 + cost2, cost1s, cost2s, hP,hA,dP,dA,hBP,hBA
    end
end                    

JJ

In [74]:
# Quick test of JJ with multiple rule_and_delay_periods

if !isdefined(:plot_PA)
    include("pro_anti.jl")
end

fname = "MiniOptimized/farm_C17_Farms024_0049.jld"

mypars, extra_pars, args, params = load(fname,"mypars", "extra_pars", "args", "pars3")

new_pars = Dict(
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1, 1.2],
:target_period          =>          0.4,
:target_periods         =>          [0.4],
:post_target_period     =>          0.0001,
:post_target_periods    =>          [0.0001],
)

mypars = merge(merge(mypars, extra_pars), new_pars)

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, 
proValls, antiValls, opto_fraction, pro_input, anti_input = 
JJ(20, 20; verbose=true, model_details=true, make_dict(args, params, mypars)...)

size(proValls)

Opto condition # 1
     - 1 - cost=0.0830004, cost1=0.0833324, cost2=-0.000331953
     - 1 - mean(hitsP)=1, mean(diffsP)=0.999928 mean(hitsA)=4.0488e-06, mean(diffsA)=0.99179
Opto condition # 2
     - 2 - cost=0.0450833, cost1=0.0454167, cost2=-0.000333328
     - 2 - mean(hitsP)=1, mean(diffsP)=0.999991 mean(hitsA)=6.94514e-14, mean(diffsA)=0.999974
Opto condition # 3
     - 3 - cost=0.0829815, cost1=0.0832695, cost2=-0.000288008
     - 3 - mean(hitsP)=0.999977, mean(diffsP)=0.9006 mean(hitsA)=0.000270293, mean(diffsA)=0.827449
OVERALL
     -- cost=0.0703551, cost1=0.0706729, cost2=-0.000317763


(3, 2)

In [53]:
0.05*0.05/3

0.0008333333333333335

In [44]:
# Quick test of JJ

if !isdefined(:plot_PA)
    @printf("Loading pro_anti.jl\n")
    include("pro_anti.jl")
end

fname = "MiniOptimized/farm_C17_Farms024_0049.jld"

mypars, extra_pars, args, params = load(fname, "mypars", "extra_pars", "args", "pars3")

# extra_pars[:seedrand] = Int64(round(time()*1000))

cost, cost1s, cost2s = JJ(800, 800; verbose=true, verbose_file=STDOUT, make_dict(args, params+0.000001*rand(12,1), 
    merge(mypars, extra_pars))...)

Opto condition # 1
     - 1 - cost=-0.000829907, cost1=9.0088e-05, cost2=-0.000919995
     - 1 - mean(hitsP)=0.89636, mean(diffsP)=0.94104 mean(hitsA)=0.71292, mean(diffsA)=0.898951
Opto condition # 2
     - 2 - cost=-0.000623646, cost1=0.000208878, cost2=-0.000832523
     - 2 - mean(hitsP)=0.839249, mean(diffsP)=0.872255 mean(hitsA)=0.482617, mean(diffsA)=0.792791
Opto condition # 3
     - 3 - cost=-0.000765945, cost1=0.000121763, cost2=-0.000887708
     - 3 - mean(hitsP)=0.902569, mean(diffsP)=0.963889 mean(hitsA)=0.715392, mean(diffsA)=0.811527
OVERALL
     -- cost=-0.000739833, cost1=0.000140243, cost2=-0.000880076


(-0.0007398327632456121, [9.0088e-5; 0.000208878; 0.000121763], [-0.000919995; -0.000832523; -0.000887708], [0.89636; 0.839249; 0.902569], [0.71292; 0.482617; 0.715392], [0.94104; 0.872255; 0.963889], [0.898951; 0.792791; 0.811527], [0.895; 0.84375; 0.90125], [0.715; 0.48125; 0.715])

In [25]:
extra_pars[:cbeta]

0.001

In [23]:
func1 = (;params...) ->  JJ(20, 20; # mypars[:nPro], mypars[:nAnti]; 
    verbose=true, merge(merge(mypars, extra_pars), Dict(params))...)[1]


val, grad, hess = keyword_vgh(func1, args, params)

Opto condition # 1
     - 1 - cost=0.00242325, cost1=0.00273568, cost2=-0.000312429
     - 1 - mean(hitsP)=1, mean(diffsP)=0.997967 mean(hitsA)=0.780088, mean(diffsA)=0.876609
Opto condition # 2
     - 2 - cost=0.00144395, cost1=0.00172656, cost2=-0.000282616
     - 2 - mean(hitsP)=0.94961, mean(diffsP)=0.963352 mean(hitsA)=0.520912, mean(diffsA)=0.732343
Opto condition # 3
     - 3 - cost=0.00170228, cost1=0.00200597, cost2=-0.000303692
     - 3 - mean(hitsP)=1, mean(diffsP)=0.999609 mean(hitsA)=0.74512, mean(diffsA)=0.822541
OVERALL
     -- cost=0.00185649, cost1=0.00215607, cost2=-0.000299579


(0.001856491708603753, [0.0336936, -0.0161271, -0.0204646, 0.016965, 0.00230334, -0.0889022, -0.00870617, -0.0180517, -0.00200409, -0.036098, 0.000982461, -0.0515996], [-6.35116 -0.20434 … 0.0 0.183644; -0.20434 0.0528846 … -0.236639 0.44817; … ; 0.0 -0.236639 … -0.282047 -0.482861; 0.183644 0.44817 … -0.482861 -2.77429])

In [ ]:
model_params[:opto_periods]

In [ ]:
args = ["vW", "hW", "sW", "dW", "anti_rule_strength", "pro_rule_strength", "constant_excitation",
    "const_pro_bias", "U_rest", "theta"] # , "beta", "tau", "right_light_excitation", "opto_strength"]
pars = vectorize_dict(model_params, args)

func = (;params...) -> JJ(100, 100; plot_list=[1:10;], plot_conditions=[false,false,false,false, false], verbose=true, 
merge(model_params, Dict(params))...)[1]

# Julia 0.5.2 takes 43 secs for next line
# Julia 0.6 takes 43 secs for next line
value, grad, hess = @time(keyword_vgh(func, args, pars))

# Julia 0.5.2 takes 26 secs for next line
# Julia 0.6 takes 26 secs for next line
value, grad, hess = @time(keyword_vgh(func, args, pars))



# Defining load_run()

In [ ]:
#@include_me  pro_anti.jl


"""
model_params, F, nPro, nAnti = load_run(run_name; farmdir="FarmFields")

Loads a run and sets everything into a self-contained model_params so that you could then run it directly:

    JJ(model_params[:nPro], model_params[:nAnti]; model_params...);

If the model_params for the run included :start_pro and :start_anti entries (or "start_pro" and
"start_anti" entries in F), it uses those. Otherwise
sets :start_pro and :start_anti to a default of [-0.5, -0.5, -0.5, -0.5]


# PARAMETERS:

- run_name   A string representing the run. If it doesn't end in .mat, the .mat is added to it

# OPTIONAL PARAMETERS:

- farmdir   A string representing the directory in which the run is found

# RETURNS:

- model_params    The dictionary with all necessary params

- F               A dictionary with the raw matread of the run file

- nPro            equals model_params[:nPro]

- nAnti           equals model_params[:nAnti]



"""
function load_run(run_name; farmdir="FarmFields")

    default_U_start = [-0.5, -0.5, -0.5, -0.5]
    
    if !endswith(run_name, ".mat")
        run_name = run_name * ".mat"
    end
    
    F = matread(farmdir * "/" * run_name)
    model_params = symbol_key_ize(F["model_params"])
    model_params[:rule_and_delay_periods] = F["rule_and_delay_periods"]
    model_params[:rule_and_delay_period]  = model_params[:rule_and_delay_periods][1]
    model_params[:target_period]          = model_params[:target_period]
    model_params[:post_target_periods]    = F["post_target_periods"]
    model_params[:post_target_period]     = model_params[:post_target_periods][1]
    model_params[:seedrand]=F["test_sr"]
    model_params[:cbeta] =F["cb"]
    if ~haskey(model_params, :start_pro)
        if ~haskey(F, "start_pro")
            model_params[:start_pro] = default_U_start
        else
            model_params[:start_pro] = F["start_pro"]
        end
    end
    if ~haskey(model_params, :start_anti)
        if ~haskey(F, "start_anti")
            model_params[:start_anti] = default_U_start
        else
            model_params[:start_anti] = F["start_antia"]
        end
    end
    model_params = make_dict(F["args"], F["pars"], model_params)

    nPro = model_params[:nPro]
    nAnti = model_params[:nAnti]
    
    if ~haskey(model_params, :target_periods)
        model_params[:target_periods] = 0.1  # Use the JJ() default
        model_params[:target_period]  = 0.1  # Use the JJ() default
        @printf("\n\nWARNING: :target_periods was not specified, JJ() would ignore :target_period and\n")
        @printf("would use a default of :target_periods=>[0.1].  Setting that explicitly here, no warning from JJ()\n")
        @printf("will be elicited.\n\n")
    end

    
    return model_params, F, nPro, nAnti
end

## Example of using load_run() and then JJ()

In [ ]:
model_params, F, nPro, nAnti = load_run("farm_LD0003");

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls = JJ(nPro, nAnti; verbose=true, 
    model_details=true, model_params...);

In [ ]:
model_params, F, nPro, nAnti = load_run("farm_LA0003", farmdir="../for_carlos_without_runs/goodfarms");

JJ(10, 10; plot_list=[1:10;], plot_conditions=[false, false, false, true, false], verbose=true, model_params...);
# JJ(10, 10; plot_list=[1:10;], plot_conditions=[true, false, false, false, false], verbose=true, model_params...);
# JJ(30, 30; plot_list=[1:30;], plot_conditions=[false, false, false, false, true], verbose=true, model_params...);


In [ ]:
model_params[:opto_periods]

In [ ]:
G = matread("goodfarms_new/good0040.mat")

In [ ]:
model_params, F, nPro, nAnti = load_run("farm_LD0003");


# plotting farm runs

In [ ]:
#@include_me  farm_plotting_carlos.jl

# for fname in filter(x -> startswith(x, "farm_"), readdir("goodfarms"))


"""
plot_farm(fname, farmdir="goodfarm")

Loads and runs ten trials of a farm that is assumed to have 5 different opto conditions in its
opto_periods (control, full, rule, delay, target/choice). Plots in figure 1 ten trials from each
of control, delay, and target/choice
"""
function plot_farm(fname, farmdir="goodfarm")
    model_params, F, nPro, nAnti = load_run(fname, farmdir="goodfarms");


    figure(1); clf();
    ax_set = Dict("pro_Vax"=>subplot(5,3,1), "pro_Dax"=>subplot(5,3,4), 
        "anti_Vax"=>subplot(5,3,10), "anti_Dax"=>subplot(5,3,13))
    proVs = antiVs, proFullVs, antiFullVs = 
        run_ntrials(10, 10; plot_list=[1:10;], opto_times=model_params[:opto_periods][1,:], 
        ax_set=ax_set, profig=1, antifig=1, clearfig=false, plot_Us=false, model_params...);
    
    ax_set = Dict("pro_Vax"=>subplot(5,3,2), "pro_Dax"=>subplot(5,3,5), 
        "anti_Vax"=>subplot(5,3,11), "anti_Dax"=>subplot(5,3,14))
    proVs, antiVs, proFullVs, antiFullVs = 
        run_ntrials(10, 10; plot_list=[1:10;], opto_times=model_params[:opto_periods][4,:], 
        ax_set=ax_set, profig=1, antifig=1, clearfig=false, plot_Us=false, model_params...);

    ax_set = Dict("pro_Vax"=>subplot(5,3,3), "pro_Dax"=>subplot(5,3,6), 
        "anti_Vax"=>subplot(5,3,12), "anti_Dax"=>subplot(5,3,15))
    proVs, antiVs, proFullVs, antiFullVs = 
          run_ntrials(10, 10; plot_list=[1:10;], opto_times=model_params[:opto_periods][5,:], 
        ax_set=ax_set, profig=1, antifig=1, clearfig=false, plot_Us=false, model_params...);
    

    subplot(5,3,1); title("(control)")
    subplot(5,3,2); title("PRO (delay)")
    subplot(5,3,3); title("(target")
    subplot(5,3,11); title("ANTI")
end


function histoit(F)
    hP = zeros(5,1)
    hA = zeros(5,1)

    for i=1:5,
        VR = F["runs_pro"][:,end,:,i,1][1,:]
        VL = F["runs_pro"][:,end,:,i,1][4,:]

        hP[i] = mean(sign(VR-VL))

        VR = F["runs_anti"][:,end,:,i,1][1,:]
        VL = F["runs_anti"][:,end,:,i,1][4,:]

        hA[i] = mean(sign(VL-VR))
    end
    figure(7); clf()
    bar(1:3, hP[[1,4,5]], 0.25)
    bar((1:3)+0.25, hA[[1,4,5]], 0.25)
end

### example of plotting a farm

In [ ]:
pygui(true)
plot_farm("farm_LD0032");


In [ ]:
#@include_me  farm_plotting_carlos.jl

pygui(true)

for resname in filter(x -> startswith(x, "res_"), readdir("../for_carlos_without_runs/goodfarms"))
    F = matread("../for_carlos_without_runs/goodfarms/" * resname)
    # histoit(F)
    fname = F["orig_file"]
    plot_farm(fname)
    
    @printf("==> This was %s\n\n", fname)
    @printf("Type anything to go on to next run, type q to quit\n")
    q = chomp(readline())
    if startswith(q, "q") || startswith(q, "Q")
        break
    end
end

In [ ]:
model_params, F, nPro, nAnti = load_run("farm_LA0003", farmdir="goodfarms");

cost, cost1s, cost2s, hP, hA, proVall, antiVall = JJ(nPro, nAnti; verbose=true, model_details=true, model_params...);



In [ ]:
figure(1); subplot(3,1,1) title("PRO -- delay")

In [ ]:
size(proVall[3])

# ---END OF CODE FOR LIBRARY AND USE BY OTHERS --

In [ ]:
#@include_me   farming.jl

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   1,   0.2,  1,    0.39,                0.15,                       0.1]

args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]


# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2])

model_params = merge(model_params, Dict(:post_target_period=>0.5))
# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]


# ======== SEARCH ZONE:

sbox = Dict(:sW=>[0.001 0.5], :vW=>[-0.5 0.5], :hW=>[-0.5 0.5], :dW=>[-0.5 0.5],
:constant_excitation=>[-0.5 0.5], :right_light_excitation=>[0.1 0.5], :target_period_excitation=>[0.1 0.5],
:const_pro_bias=>[0 0.2], :sigma=>[0.02 0.19])

cbetas = [0.01]

fbasename = "FarmFields/farm_TESTING_"

while true
    sr = convert(Int64, round(time()))

    myseed = copy(seed);
    for i=1:length(args)
        sym = Symbol(args[i])
        if haskey(sbox, sym)
            myseed[i] = sbox[sym][1] + diff(sbox[sym],2)[1]*rand()
        end
    end
    nPro=100; nAnti=100

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.1, 0.3]
    target_periods         = [0.1]

    theta1 = 0.15; theta2 = 0.25

    for cb in cbetas
        func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
        target_periods = target_periods, theta1=theta1, theta2=theta2,
            post_target_periods=post_target_periods,
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)[1]
        
        # And at the standard cb=0.01 for comparison to other cbs
        # standard_func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
        #    theta1=theta1, theta2=theta2,
        #    post_target_periods=post_target_periods,
        #    seedrand=sr, cbeta=0.01, verbose=false, merge(model_params, Dict(params))...)[1]        
                
        @printf("Going with seed = "); print_vector_g(myseed); print("\n")
        pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(myseed, args, bbox, func,
            start_eta = 0.01, tol=1e-12, verbose=true, verbose_every=2, maxiter=400)
        @printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

        value, grad, hess = keyword_vgh(func, args, pars)
        
        myfilename = next_file(fbasename, 4)

        matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "pars"=>pars, "traj"=>traj,
        "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>nPro, "nAnti"=>nAnti, "sr"=>sr, "cb"=>cb,
        "theta1"=>theta1, "theta2"=>theta2,
        "scost"=>scost, "value"=>value, "grad"=>grad, "hess"=>hess,
        "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox),
        "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods))

    end
end


# Running a ProAnti optimization  

This is the code that ran farm F


In [ ]:
#@include_me   farming.jl

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   1,   0.2,  1,    0.39,                0.15,                       0.1]

args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]


# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2])

model_params = merge(model_params, Dict(:post_target_period=>0.5))
# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]


# ======== SEARCH ZONE:

sbox = Dict(:sW=>[0.001 0.5], :vW=>[-0.5 0.5], :hW=>[-0.5 0.5], :dW=>[-0.5 0.5],
:constant_excitation=>[-0.5 0.5], :right_light_excitation=>[0.1 0.5], :target_period_excitation=>[0.1 0.5],
:const_pro_bias=>[0 0.2], :sigma=>[0.02 0.19])

cbetas = [0.02, 0.04]

fbasename = "FarmFields/farm_TESTING_"

while true
    myseed = seed;
    sr = convert(Int64, round(time()))

    myseed = copy(seed);
    for i=1:length(args)
        sym = Symbol(args[i])
        if haskey(sbox, sym)
            myseed[i] = sbox[sym][1] + diff(sbox[sym],2)[1]*rand()
        end
    end
    nPro=10; nAnti=10

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    theta1 = 0.15; theta2 = 0.25

    for cb in cbetas
        func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            theta1=theta1, theta2=theta2,
            post_target_periods=post_target_periods,
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)[1]
        
        # And at the standard cb=0.01 for comparison to other cbs
        standard_func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            theta1=theta1, theta2=theta2,
            post_target_periods=post_target_periods,
            seedrand=sr, cbeta=0.01, verbose=false, merge(model_params, Dict(params))...)[1]        
                
        @printf("Going with seed = "); print_vector_g(myseed); print("\n")
        pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(myseed, args, bbox, func,
            start_eta = 0.01, tol=1e-12, verbose=true, verbose_every=10, maxiter=400)
        @printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

        value, grad, hess = keyword_vgh(func, args, pars)
        scost = standard_func(;make_dict(args, pars, model_params)...)
        
        myfilename = next_file(fbasename, 4)

        matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "pars"=>pars, "traj"=>traj,
        "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>nPro, "nAnti"=>nAnti, "sr"=>sr, "cb"=>cb,
        "theta1"=>theta1, "theta2"=>theta2,
        "scost"=>scost, "value"=>value, "grad"=>grad, "hess"=>hess,
        "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox),
        "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods))

    end
end


### Example of taking the gradient and Hessian of the cost function JJ()

In [ ]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias"]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]

func = (;params...) -> JJ(100, 10; rule_and_delay_periods = [0.4, 0.8], seedrand=30, cbeta=0.01, 
plot_list = [], verbose=false, merge(model_params, Dict(params))...)[1]

cost, grad, hess = keyword_vgh(func, args, seed)

# func(;make_dict(args, seed+ [1,0.2,0,0,0,0,0])...) - func(;make_dict(args, seed)...)


### Looking into farm animals

In [ ]:
# Finding the best farm animal

fnames = filter(x -> startswith(x, "farm_L"), readdir("FarmFields/"))
costs = zeros(size(fnames))
for i=1:length(fnames)
    G = matread("FarmFields/" * fnames[i])
    costs[i] = G["cost"]
end

find(costs .== minimum(costs))

In [ ]:
merge(make_dict(G["args"], G["pars"], model_params), Dict(:sigma=>0.2))

In [ ]:
make_dict(G["args"], G["pars"])

In [ ]:
epochs = ["control"; "full"; "rule"; "delay"; "target"]

In [ ]:
pygui(true); figure(1)

# mixed_sign_hessian_farm_F_analysis.jl  -- Looking into Hessians with mixed-sign eigenvalues

**RESULT: The mixed-sign Hessian eigenvalues are due to the walls**

There are 122 farm F animals that stopped before their 400-iteration limit.  Of those, if we look at the Hessian for only those parameters that had not reached a bounding wall, all but one farm animal had all-positive eigenvalues (including F_0564). [Thus, for example, for a file in which 2 parameters had reached a bounding wall and 7 parameters had not, we're looking at a 7x7 Hessian.]

The single example with mixed-sign eigenvalues is F_0534. The solution in this farm animal is not terrible, but it ranks as the 203rd best solution out of 751 (27th percentile), far from what we would usually consider. If we took the top 20% solutions we wouldn't even be considering this lone example.

Given that, I think I'm going to proceed working on the assumption that for most files, the walls are the issue; the rare files for which that is not true I propose we just toss, and if those become not rare in the future then I will revisit.

Code: [`mixed_sign_hessian_farm_F_analysis.jl`](http://localhost:8888/edit/mixed_sign_hessian_farm_F_analysis.jl)




In [ ]:
#@include_me   mixed_sign_hessian_farm_F_analysis.jl

"""
walled, free = id_pars_at_walls(bbox, args, pars; tol=1e-3)

Returns a list of elements in pars that are within tol of their bounds described in the bbox Dict(),
and a list of those not within tol of their bounds.  The union of walled and free will be equal to 1:length(pars).

"""
function id_pars_at_walls(bbox, args, pars; tol=1e-3)
    walled = Array{Int}(0)
    free   = Array{Int}(0)
    i = 1; while i<=length(args)
        if typeof(args[i])<:String && haskey(bbox, Symbol(args[i]))
            range = bbox[Symbol(args[i])]
            if any(abs(pars[i]-range).<tol)
                walled = [walled; i]
            else
                free = [free; i]
            end    
        elseif typeof(args[i])<:String
            free = [free; i]
        elseif haskey(bbox, Symbol(args[i][1]))
            range = bbox[Symbol(args[i][1])]
            for j=1:args[i][2]
                if any(abs(pars[i+j-1]-range).<tol)
                    walled = [walled; i+j-1]
                else
                    free = [free; i+j-1]
                end    
            end
            i = i+args[i][2]-1
        else
            for j=1:args[i][2]; free = [free; i+j-1]; end
            i = i+args[i][2]-1
        end
    i=i+1; end    
    
    return walled, free
end


walled, free = id_pars_at_walls(Dict(:a=>[1, 2], :b=>[0.1, 0.2]), ["c", "a", ["b" 2]], [10, 1.9999, 0.1, 0.15])

walled, free = id_pars_at_walls(bbox, args, pars; tol=1e-1)

In [ ]:
#@include_me   mixed_sign_hessian_farm_F_analysis.jl

# ========  This is copied from the code for farm F, telling us how it was run ==============================

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   1,   0.2,  1,    0.39,                0.15,                       0.1]

args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]


# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2])

model_params = merge(model_params, Dict(:post_target_period=>0.5))
# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]


# ======== SEARCH ZONE:

sbox = Dict(:sW=>[0.001 0.5], :vW=>[-0.5 0.5], :hW=>[-0.5 0.5], :dW=>[-0.5 0.5],
:constant_excitation=>[-0.5 0.5], :right_light_excitation=>[0.1 0.5], :target_period_excitation=>[0.1 0.5],
:const_pro_bias=>[0 0.2], :sigma=>[0.02 0.19])

# ========  END  --   This is copied from the code for farm F, telling us how it was run ===================

# Now read all the files to get all there costs and iteration numbers

fnames = filter(x -> startswith(x, "farm_F"), readdir("FarmFields/"))
scosts = zeros(size(fnames))
niters = zeros(size(fnames))
for i=1:length(fnames) 
    A = matread("FarmFields/" * fnames[i])  
    scosts[i] = A["scost"]
    niters[i] = size(A["traj"],2)
end

pygui(true)
figure(1); clf();
plot(scosts, niters, ".")
xlabel("cost at standard beta=0.01")
ylabel("number of iterations run")

In [ ]:
#@include_me   mixed_sign_hessian_farm_F_analysis.jl


# #################  FINALLY TO THE ANALYSIS PART   ################

goodies = find((niters.<400)) #  & (scosts.<-0.007))
mixed_sign = Array{String}(0)

for i=1:length(goodies)
    A = matread("FarmFields/" * fnames[goodies[i]])

    args = A["args"]
    pars = A["pars"]
    nPro = A["nPro"]
    nAnti = A["nAnti"]
    rule_and_delay_periods = A["rule_and_delay_periods"]
    post_target_periods    = A["post_target_periods"]
    theta1 = A["theta1"]
    theta2 = A["theta2"]
    sr = A["sr"]
    cb = A["cb"]
    sbox = symbol_key_ize(A["sbox"])
    bbox = symbol_key_ize(A["bbox"])
    model_params = symbol_key_ize(A["model_params"])


    func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
        theta1=theta1, theta2=theta2,
        post_target_periods=post_target_periods,
        seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)[1]

    standard_func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
        theta1=theta1, theta2=theta2,
        post_target_periods=post_target_periods,
        seedrand=sr, cbeta=0.01, verbose=false, merge(model_params, Dict(params))...)[1]


    value, grad, hess = keyword_vgh(func, args, pars)

    walled, free = id_pars_at_walls(bbox, args, pars)
    L, V = eig(hess[free,free])
    @printf("%d/%d: file %s had %d free parameters, with hessian eigenvalues ", 
        i, length(goodies), fnames[goodies[i]], length(free))
    print_vector_g(L); print("\n")
    
    if any(L.<0)
        @printf("\n    *** File %s had off-wall mixed sign eigenvalues!\n\n", fnames[goodies[i]])
        mixed_sign = [mixed_sign ; fnames[goodies[i]]]
    end
end



In [ ]:
find(fnames.=="farm_F_0534")
scosts[534]

In [ ]:
A = matread("FarmFields/farm_F_0007")

args = A["args"]
pars = A["pars"]
nPro = A["nPro"]
nAnti = A["nAnti"]
rule_and_delay_periods = A["rule_and_delay_periods"]
post_target_periods    = A["post_target_periods"]
theta1 = A["theta1"]
theta2 = A["theta2"]
sr = A["sr"]
cb = A["cb"]
sbox = symbol_key_ize(A["sbox"])
bbox = symbol_key_ize(A["bbox"])
model_params = symbol_key_ize(A["model_params"])


func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
    theta1=theta1, theta2=theta2,
    post_target_periods=post_target_periods,
    seedrand=sr, cbeta=cb, verbose=true, merge(model_params, Dict(params))...)[1]

standard_func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
    theta1=theta1, theta2=theta2,
    post_target_periods=post_target_periods,
seedrand=sr, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...)[1]


value, grad, hess = keyword_vgh(func, args, pars)


In [ ]:
A = matread("FarmFields/farm_F_0534")

args = A["args"]
pars = A["pars"]
nPro = A["nPro"]
nAnti = A["nAnti"]
rule_and_delay_periods = A["rule_and_delay_periods"]
post_target_periods    = A["post_target_periods"]
theta1 = A["theta1"]
theta2 = A["theta2"]
sr = A["sr"]
cb = A["cb"]
sbox = symbol_key_ize(A["sbox"])
bbox = symbol_key_ize(A["bbox"])
model_params = symbol_key_ize(A["model_params"])


func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
    theta1=theta1, theta2=theta2,
    post_target_periods=post_target_periods,
    seedrand=sr, cbeta=cb, verbose=true, merge(model_params, Dict(params))...)[1]

standard_func =  (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
    theta1=theta1, theta2=theta2,
    post_target_periods=post_target_periods,
seedrand=sr, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...)[1]


value, grad, hess = keyword_vgh(func, args, pars)


In [ ]:
length(find(scosts.<scosts[534]))/length(scosts)

In [ ]:
length(scosts)

In [ ]:
bbox

In [ ]:
A

In [ ]:
GG = matread("error_report.mat")
hess = GG["hess"]; eta = GG["eta"]; grad = GG["grad"]
cpm_out = constrained_parabolic_minimization(hess, grad'', eta, maxiter=500, tol=1e-20, do_plot=true, verbose=true)                


# Alex's farm G

In [ ]:
#@include_me alex_farm_g0011.jl

include("pro_anti.jl")

A = matread("FarmFields/farm_G_1.mat0011")

model_params = symbol_key_ize(A["model_params"])
for k in keys(A); print(k); print("  "); end

A

In [ ]:
[args[:]  A["myseed"]]

In [ ]:
#@include_me alex_farm_g0011.jl

bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2]);

model_params = symbol_key_ize(A["model_params"])

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    standard_func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
    rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
    post_target_periods=post_target_periods,  seedrand=A["sr"], cbeta=0.01, verbose=true, 
    merge(model_params, Dict(params))...)[1]        

value, grad, hess = keyword_vgh(standard_func, A["args"], A["myseed"])


In [ ]:
#@include_me alex_farm_g0011.jl

pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(A["myseed"], A["args"], bbox, standard_func,
        start_eta = 0.03, tol=1e-18, verbose=true, verbose_every=1, maxiter=4000)


In [ ]:
traj

In [ ]:

fnames = readdir("FarmFields/")
for f in filter(x -> startswith(x, "farm_G_1.mat"), readdir("FarmFields/"))

    A = matread("FarmFields/" * f)
    model_params = symbol_key_ize(A["model_params"])

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    standard_func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
    rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
    post_target_periods=post_target_periods,  seedrand=A["sr"], cbeta=0.01, verbose=true, 
    merge(model_params, Dict(params))...)[1]        

    standard_func(;make_dict(A["args"], A["pars"])...)

    @printf("File %s, niters was %d\n\n", f, size(A["traj"],2))
end

# Sandbox -- the in flux crazy wilds from here on

various things in progress

In [ ]:
next_file("FarmFields/farm_E_", 4)

In [ ]:
"""
cost = standard_cost(args, pars, sr, model_params)

Computes the cost as if cb had been 0.01 and theta1=0.15 and theta2=0.25
"""
function standard_cost(args, pars, sr, model_params)
    cb = 0.01
    theta1 = 0.15
    theta2 = 0.25
    nPro=100; nAnti=100

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    func = (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            post_target_periods=post_target_periods,
            theta1=theta1, theta2=theta2, 
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)
    
    return func(;make_dict(args, pars)...)
end
    
"""
cost = standard_cost(filename)

Returns the standard cost (at cb=0.01, theta1=0.15, theta2=0.25) and inserts it into the file with 
key "scost2" if it wasn't there already
"""
function standard_cost(filename; verbose=false)
    A = matread(filename)
    if !haskey(A, "scost2")
        get!(A, "scost2", standard_cost(A["args"], A["pars"], A["sr"], symbol_key_ize(A["model_params"])))
        if verbose
            @printf("File %s did not have scost, adding its value %g\n", filename, A["scost"])
        end
        matwrite(filename, A)
    end
    return A["scost"]
end

#####################################


"""
value, grad, hess = standard_vgh(args, pars, sr, model_params)

Computes the value, gradient, and hessian as if cb had been 0.01 and theta1=0.15 and theta2=0.25
"""
function standard_vgh(args, pars, sr, model_params; theta1=0.025, theta2=0.035)
    cb = 0.01
    nPro=100; nAnti=100

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    func = (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            post_target_periods=post_target_periods,
            theta1=theta1, theta2=theta2, 
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    value, grad, hess = keyword_vgh(func, args, pars)
    
    return value, grad, hess
end

func = (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods, verbose=true,
            post_target_periods=post_target_periods,
            theta1=theta1, theta2=theta2,
            seedrand=sr, cbeta=cb, verbose=false, merge(my_params, Dict(params))...)[1]


"""
value, grad, hess = standard_vgh(filename)

Returns the standard cost (at cb=0.01, theta1=0.15, theta2=0.25), gradient, and hessian
and inserts them into the file with key "scost2", "grad" and "hess" if they weren't there already
"""
function standard_vgh(filename; verbose=false, force=false)
    A = matread(filename)
    if !haskey(A, "theta1"); get!(A, "theta1", 0.025); end
    if !haskey(A, "theta2"); get!(A, "theta2", 0.035); end
    if !haskey(A, "rule_and_delay_periods"); get!(A, "rule_and_delay_periods", [0.4, 1.2]); end;
    if !haskey(A, "post_target_periods");    get!(A, "post_target_periods",    [0.5, 1.5]); end;
      
    if force || !haskey(A, "value") || !haskey(A, "grad")  || !haskey(A, "hess")
        value, grad, hess = standard_vgh(A["args"], A["pars"], A["sr"], 
        symbol_key_ize(A["model_params"]); theta1=A["theta1"], theta2=A["theta2"]) 
        if !haskey(A, "value"); get!(A, "value", value); else A["value"] = value; end;
        if !haskey(A, "grad"); get!(A, "grad", grad); else A["grad"] = grad; end;
        if !haskey(A, "hess"); get!(A, "hess", hess); else A["hess"] = hess; end;
        if verbose
            @printf("File %s did not have value or grad or hess, adding its value %g\n", filename, A["value"])
        end
        matwrite(filename, A)
    end
    return A["value"], A["grad"], A["hess"]
end



In [ ]:
# A = matread("FarmFields/farm_E_0001")
# value, grad, hess = standard_vgh(A["args"], A["pars"], A["sr"], symbol_key_ize(A["model_params"]))
standard_vgh("FarmFields/farm_A_0001"; verbose=true, force=true)
A = matread("FarmFields/farm_A_0001")


In [ ]:
eval(Symbol("hess"))

In [ ]:
standard_vgh("FarmFields/farm_A_0001", verbose=true)
A = matread("FarmFields/farm_A_0001")
A

In [ ]:
A = matread("farm_A_0001")
@printf("At cb=0.002, cost was %g. At cb=0.01, cost was %g\n", A["cost"], standard_cost("farm_A_0001"))


### Run the next three cells, in sequence, to see results of an individual farm animal and to see computation of gradient and hessian of the result

In [ ]:
A = matread("FarmFields/farm_G_1.mat0001")
model_params = symbol_key_ize(A["model_params"])
args         = A["args"];
nAnti        = A["nAnti"]
nPro         = A["nPro"]
pars         = A["pars"]
sr           = A["sr"]
traj         = A["traj"]
cpm_traj     = A["cpm_traj"]
cb           = A["cb"]
theta1       = A["theta1"]
theta2       = A["theta2"]
rule_and_delay_periods = A["rule_and_delay_periods"]
post_target_periods    = A["post_target_periods"]


@printf("Cost went from %g to %g (at standard beta=0.01, cost ended at %g); max iters on cpm was %d\n\n", 
traj[2,1], traj[2,end], A["scost"], maximum(cpm_traj[1,:]))

A

In [ ]:
good_seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]

[args pars good_seed]

In [ ]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 0.2; 0.4; 0.02], model_params)

run_factor = 2

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; verbose=true, seedrand=sr, rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


# A function that takes only keyword-value pairs and returns a scalar. Its default parameter values are the result
# of the farm (as put into my_params, which hold the farm's output)
func = (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods, verbose=true,
            post_target_periods=post_target_periods,
            theta1=theta1, theta2=theta2,
            seedrand=sr, cbeta=cb, verbose=false, merge(my_params, Dict(params))...)[1]

# keyword_vgh takes that kind of function
cost, grad, hess = keyword_vgh(func, args, pars)



In [ ]:
traj[:,end]

### Evaluating the results across many farm animals

In [ ]:
basename = "farm_G_1.mat"
fnames = readdir("FarmFields")
matched_filenames = Array{Bool}(length(fnames))
for i=1:length(fnames)
    matched_filenames[i] = ismatch(Regex(@sprintf("^%s", basename)), fnames[i])
end
myguys = fnames[find(matched_filenames)]
costs  = zeros(size(myguys))
cbs    = zeros(size(myguys))
nsteps = zeros(size(myguys)) 
for i=1:length(myguys)
    A = matread("FarmFields/" * myguys[i])
    costs[i]  = A["cost"] # standard_cost(myguys[i]; verbose=true)
    cbs[i]    = A["cb"]
    nsteps[i] = size(A["traj"],2)
end


cbset = unique(cbs)

figure(3); clf();
for i=1:length(cbset)    
    ax1 = subplot(length(cbset),1,i)
    h1 = plt[:hist](costs[find(cbs.==cbset[i])]) # , -0.02:0.001:0.008);
    ylabel("# of runs")
    title(@sprintf("cb=%g  (%d total)", cbset[i], length(find(cbs.==cbset[i]))))
    if i < length(cbset)
        remove_xtick_labels(ax1)
    end
    if i==length(cbset)
        xlabel("final cost")
    end
end
        
unique(cbs)

In [ ]:
figure(3); clf()
plt[:hist](nsteps, 0:2:400);

In [ ]:
A = matread("FarmFields/farm_G_1.mat0001")

# Example of 9-param, including diagonal weight and sigma, and robust across 3x in rule/delay period and post-target period

In [ ]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "sigma"]
seed = [0.2,   -1.7, -1.7,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   0.17,  0.17,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      -0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                     0.1]
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :sigma=>[0.05 1])
model_params = merge(model_params, Dict(:post_target_period=>0.5))


# Now with constant_pro_bias and a fixed sigma=0.1
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   -1.7, -1.7,  0,    0.39,                0.15,                       0.1]
args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]
model_params = merge(model_params, Dict(:post_target_period=>0.5))
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.25])

# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]

# ==========

nPro=100; nAnti=100

rule_and_delay_periods = [0.4, 1.2]
post_target_periods    = [0.5, 1.5]

pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(seed, args, bbox, 
(;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
post_target_periods=post_target_periods,
seedrand=31, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...),
start_eta = 0.01, tol=1e-9, verbose=true)

pars'

In [ ]:
[args pars]

In [ ]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 1.5; 1; 0.01], model_params)

run_factor = 10

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


# Example of a ProAnti network optimization

This one uses two rule periods and two post_target_periods to try to get some stability in the trained network.

In [ ]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "sigma"]
seed = [0.2,   -1.7, -1.7,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   0.17,  0.17,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      -0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :sigma=>[0.05 1])
model_params = merge(model_params, Dict(:post_target_period=>0.5))


# Now with constant_pro_bias and a fixed sigma=0.1
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias"]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]
model_params = merge(model_params, Dict(:post_target_period=>0.5, :sigma=>0.1))
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2])

# ==========

nPro=100; nAnti=100

rule_and_delay_periods = [0.4, 0.8]
post_target_periods    = [0.5, 1]

pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(seed, args, bbox, 
(;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
post_target_periods=post_target_periods,
seedrand=31, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...),
start_eta = 0.01, tol=1e-12, verbose=true, maxiter=2)

pars'

In [ ]:
cost

In [ ]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 1.5; 0.5; 0.02], model_params)

run_factor = 10

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


In [ ]:
######################################################
#                                                    #
#         BBOX_HESSIAN_KEYWORD_MINIMIZATION          #
#                                                    #
######################################################




"""
pdict = wallwrap(bdict, pdict)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, and pdict, a dictionary of symbols
to values (or, alternatively, an Array of (Symbol, value) tuples], goes through each of the symbols in 
bdict and modifies the corresponding value in pdict putting it through a tanh so the final output lies 
within the limits in bdict.  Returns the new pdict.  Makes a copy of pdict so as not to modify the original.
"""
function wallwrap(bdict, epdict)
    local pdict = two_level_copy(epdict)
    if typeof(pdict)<:Array
        pdict = Dict(pdict)
    end

    allkeys = keys(bdict)

    for k in allkeys
        local bbox = bdict[k]
        d = 0.5*(bbox[2] - bbox[1])
        m = 0.5*(bbox[2] + bbox[1])

        pdict[k] = bbox[1] + d*(tanh((pdict[k]-m)/d)+1)
    end
    return pdict
end

    
"""
params = vector_wrap(bbox, args, eparams)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, args, an array of strings representing
symbols, and params, an array of values corresponding to the args list, puts each param that has an entry 
in bdict through the tanh-walling mechanism, and returns the result. Does not modify the contents of the 
original params vector (or bdict or args).
"""
function vector_wrap(bbox, args, eparams)
    local params = two_level_copy(eparams)
    pdict = wallwrap(bbox, make_dict(args, params))
    i=1; j=1
    for i=1:length(args)
        if typeof(args[i])<:Array
            params[j:j+args[i][2]-1] = pdict[Symbol(args[i][1])]
            j += args[i][2]-1
        else
            params[j] = pdict[Symbol(args[i])]
        end
    j = j+1
    end
    return params
end


"""
params = inverse_wall(bdict, args, wparams)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, args, an array of strings representing
symbols, and wparams, an array of values corresponding to the args list where each param that has an entry 
in bdict has alreadt been through the tanh-walling mechanism, UNwalls the ones that have a bdict entry and
returns the result. Does not modify the contents of the original params vector (or bdict or args).
"""
function inverse_wall(bdict, args, wparams)
    local params = two_level_copy(wparams)
    pdict = inverse_wall(bdict, make_dict(args, params))
    i=1; j=1
    for i=1:length(args)
        if typeof(args[i])<:Array
            params[j:j+args[i][2]-1] = pdict[Symbol(args[i][1])]
            j += args[i][2]-1
        else
            params[j] = pdict[Symbol(args[i])]
        end
        j = j+1
    end
    return params    
end

    
"""
pdict = inverse_wall(bdict, wdict)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, and wdict, a dictionary of symbols to values
(or vectors of values)  UNwalls the ones that have a bdict entry and
returns the result. Does not modify the contents of any dictionaries.
"""
function inverse_wall(bdict, wdict)
    local pdict = two_level_copy(wdict)

    allkeys = keys(bdict)
    for k in allkeys
        local bbox = bdict[k]
        d = 0.5*(bbox[2] - bbox[1])
        m = 0.5*(bbox[2] + bbox[1])

        try
            pdict[k] = m + d*0.5*log((pdict[k]-bbox[1])./(2*d - pdict[k] + bbox[1]))
        catch
            error(@sprintf("Had trouble with key %s", string(k)))
        end
    end
    return(pdict)
end
  


"""
function bbox_Hessian_keyword_minimization(seed, args, bbox, func; wallwidth=NaN, start_eta=10, tol=1e-6, 
    maxiter=400, verbose=false)

Like constrained_Hessian_minimization, but uses keyword_hessian!(). 

# PARAMETERS:

- seed        column vector, representing the starting value of the parameters.

- args        List of strings identifying parameters for differentiation, e.g., ["const_E", "w_self]

- bbox        If softbox=true (the default), should then be a Dict of Symbol=>[minval maxval] entries. An entry
            in this Dict indicates that the corresponding parameter is to be bounded, as indicated by the associated 
            [minval maxval] vector. The bbox dictionary can have fewer entries than the number of parameters, and its
            default value is Dict(), indicating an unbounded search.
                If softbox=false, then bbox should be an nargs-by-2 matrix indicating the range for each argument,
            with the minima (first column) and maxima (second column), and entries for ALL parameters.

- func        func must take only optional keyword args, and must 
            take nderivs=0, difforder=0  and declare any new matrices using ForwardDiffZeros() instead of zeros()


# OPTIONAL PARAMETERS:

- start_eta    Starting value of the radius.  It's good to start with somethibg biggish, if it is
             too much, it'll quickly get cut down.

- tol=1e-6     Numerical tolerance. If a proposed jump produces a change in func that is less than
             this, the minimization stops.

- maxiter=400  Maximum number of iterations to do before stopping

- verbose=false   If true, print out a report on each iteration of iteration number, radius size (eta),
                what type jump was proposed ("Newton" means going straight to global min, "constrained" means jump has 
                norm eta, failed means that finding the minimum at a given radius somehow didn't work). Will also
                print out the cosine of the angle between the proposed jump and the gradient.

- verbose_level   If less than 2, regular verbose output, if 2 or greater, very verbose, for debugging.

- softbox         If true, then bbox must be a Dict() and we use the tanh() mechanism for putting a fixed limit
                on the parameters.

- hardbox=false   If true, ignores wallwidth, and just rests parameter values to the bounding box if they go outside it.
                If false, adds cost function "walls" to implement the bounding box.

- walldith=NaN     Used for putting up cost function "walls" that implement the bounding box limits. Can be NaN.
                If it is NaN, then the wallwidth is a constant factor of the range width for each argument. If not NaN, must
                be an nargs-long vector that indicates the actual wall widths.

- wallwidth_factor=0.18   Only relevant if wallwidth is NaN, otherwise ignored. For each arg, the wall width
                is going to be wall_width_factor*(bbox[i,2] - bbox[i,1])


# RETURNS:

- params       A vector the size of seed that has the last values of the minimizing parameters for func
- trajectory   A (2+length(params))-by-nsteps matrix. Each column corresponds to an iteration step, and contains
                 the value of eta used, the cost, and the value of the parameters at that iteration
- cost         Final value of objective function
- cpm_traj     A 2-by-nsteps matrix, containing reports from the contrained parabolic minimization at each timestep.
             The first row is niters (how many iterations cpm's 1-d minimization ran for) and the second row is
             Dlambda, the last change in the parameter being minimized in cpm's internal search


# EXAMPLE:

```
function tester(;x=5, y=10, z=20, nderivs=0, difforder=0)
    return x^2*y + z/tanh(y)
end

params, trajectory = bbox_Hessian_keyword_minimization([0.5, 0.5], ["x", "y"], [1.1 2 ; 1.1 4], tester, 
    verbose=true, tol=1e-12, start_eta=1);
```


"""
function bbox_Hessian_keyword_minimization(seed, args, bbox, func; start_eta=0.1, tol=1e-6, maxiter=400,
    verbose=false, verbose_level=1, verbose_every=1, 
    softbox=true, hardbox=false, wallwidth=NaN, wallwidth_factor=0.18)

      
    """
    Given args, a list of string representing the arguments of interest, a bounding box for each,
    and a Symbol=>value dictionary with the corresponding parameters, computes and returns a high cost for 
    being outside the bounding box
    """
    function wall_cost(args, bbox; wallwidth=NaN, nderivs=0, difforder=0, pars...) 
        myparams = ForwardDiffZeros(length(pars), 1, nderivs=nderivs, difforder=difforder)
        pars2 = Dict()
        for i in [1:length(pars);]
            pars2[string(pars[i][1])] = pars[i][2]
        end
        for i in [1:length(args);]
            myparams[i] = pars2[args[i]]
        end
        
        if isnan(wallwidth)
            # We know that we're going to be taking hessian for params, so declare zeros accordingly:
            wallwidth = ForwardDiffZeros(length(myparams), 1, nderivs=nderivs, difforder=difforder)

            for i in [1:length(myparams);]
                wallwidth[i] = wallwidth_factor*(bbox[i,2]-bbox[i,1])
            end
        end

        retval = 0
        for i in [1:length(myparams);]
            if myparams[i]<bbox[i,1]
                retval += cosh((bbox[i,1]-myparams[i])/wallwidth[i])-1.0
            elseif bbox[i,2] < myparams[i]
                retval += cosh((myparams[i]-bbox[i,2])/wallwidth[i])-1.0                
            end
        end

        return 2*retval
    end

    traj_increment = 100
    params = 0  # Make sure to have this here so that params stays defined beyond the try/catch
    if ( !(typeof(bbox)<:Dict) ); error("Currently only supporting softbox=true, bbox must be a Dict"); end;
    try
        params = copy(inverse_wall(bbox, args, seed))
    catch y
        @printf("inverse_wall failed with error %s\n", y)
        error("Were all initial param values within the indicated walls?")
    end
    eta = start_eta
    trajectory = zeros(2+length(params), traj_increment); cpm_traj = zeros(2, traj_increment)

    if verbose
        @printf "%d: eta=%g ps=" 0 eta 
        print_vector(vector_wrap(bbox, args, params))
        @printf "\n"
    end
    
    if softbox
        if !(typeof(bbox)<:Dict); error("bhm: If softbox=true, then bbox must eb a Dict"); end
        cost, grad, hess = keyword_vgh((;pars...)->func(;wallwrap(bbox, pars)...), args, params)
    elseif hardbox
        cost, grad, hess = keyword_vgh((;pars...) -> func(;pars...), args, params)
    else
        cost, grad, hess = keyword_vgh((;pars...) -> func(;pars...) + wall_cost(args, bbox; wallwidth=wallwidth, pars...),
            args, params)        
    end
        
    chessdelta = zeros(size(params))
    
    i=0  # here so variable i is available outside the loop
    for i in [1:maxiter;]
        if i > size(trajectory, 2)
            trajectory = [trajectory zeros(2+length(params), traj_increment)]
            cpm_traj   = [cpm_traj   zeros(2, traj_increment)]
        end
        trajectory[1:2, i]   = [eta;cost]
        trajectory[3:end, i] = vector_wrap(bbox, args, params)
        
        hessdelta  = - inv(hess)*grad
        try
            if verbose && verbose_level >= 2
                @printf("bhm: about to try cpm with grad : "); print_vector_g(grad); print("\n")
                @printf("bhm:   hess :"); print_vector_g(hess[:]); print("\n");
            end
            if verbose && verbose_level >= 2
                cpm_out = constrained_parabolic_minimization(hess, grad'', eta, 
                    maxiter=500, tol=1e-20, do_plot=true, verbose=true)                
            else
                cpm_out = constrained_parabolic_minimization(hess, grad'', eta, maxiter=500, tol=1e-20)
            end
            chessdelta = cpm_out[1]; cpm_traj[1,i] = cpm_out[5]; cpm_traj[2,i] = cpm_out[6]
            jumptype = "not failed"
        catch y
            jumptype = "failed"
            if verbose
                @printf "Constrained parabolic minimization failed with error %s\n" y
                @printf "\n"
                @printf "eta was %g\n" eta
                @printf "grad was\n"
                print_vector(grad)
                @printf "\n\nhess was\n"
                for k in [1:length(grad);]
                    print_vector(hess[k,:])
                    @printf "\n"
                end
                @printf "\n"
                matwrite("error_report.mat", Dict("grad"=>grad, "hess"=>hess, "eta"=>eta))
            end
            break
        end

        if norm(hessdelta) <= eta
            new_params = params + hessdelta
            jumptype = "Newton"
        elseif jumptype != "failed" 
            new_params = params + chessdelta
            jumptype  = "constrained"
        end

        if jumptype != "failed"
            if softbox
                new_cost, new_grad, new_hess = 
                    keyword_vgh((;pars...) -> func(;wallwrap(bbox, pars)...), args, new_params)
                if verbose && verbose_level >=2
                    @printf("bhm: had new_params = : "); print_vector_g(vector_wrap(bbox, args, params)); print("\n");
                    @printf("bhm: and my bbox was : "); print(bbox); print("\n")
                    @printf("bhm: and my wallwrap output was : "); print(wallwrap(bbox, make_dict(args, new_params))); print("\n")
                    @printf("bhm: and this produced new_grad : "); print_vector_g(new_grad); print("\n")
                    @printf("bhm:   new_hess :"); print_vector_g(new_hess[:]); print("\n");                                        
                end
            elseif hardbox
                for p in [1:length(new_params);]
                    if new_params[p] < bbox[p,1]; new_params[p] = bbox[p,1]; end
                    if bbox[p,2] < new_params[p]; new_params[p] = bbox[p,2]; end
                 end        
                
                new_cost, new_grad, new_hess = keyword_vgh((;pars...) -> func(;pars...), args, new_params)
            else
                new_cost, new_grad, new_hess = keyword_vgh((;pars...) -> func(;pars...) + 
                        wall_cost(args, bbox; wallwidth=wallwidth, pars...),
                    args, new_params)                
            end
            
            if abs(new_cost - cost) < tol || eta < tol
                if verbose
                    @printf("About to break -- tol=%g, new_cost-cost=%g, eta=%g\n", tol, new_cost-cost, eta)
                end
                break
            end
        end

        if jumptype == "failed" || new_cost >= cost  
            if verbose
                @printf("eta going down: new_cost-cost=%g and jumptype='%s'\n", new_cost-cost, jumptype)
                if verbose_level >= 2
                    nwp = vector_wrap(bbox, args, new_params); wp = vector_wrap(bbox, args, params)
                    @printf("   vvv: proposed new params were : "); print_vector_g(nwp); print("\n")
                    @printf("   vvv: proposed delta params was : "); print_vector_g(nwp-wp); print("\n")
                    @printf("   vvv: grad was : "); print_vector_g(grad); print("\n")
                    costheta = dot(new_params-params, grad)/(norm(new_params-params)*norm(grad))
                    @printf("   vvv: costheta of proposed jump was %g\n", costheta)
                end
            end
            eta = eta/2
            costheta = NaN
            if eta < tol
                if verbose
                    @printf("About to break -- tol=%g, new_cost-cost=%g, eta=%g\n", tol, new_cost-cost, eta)
                end
                break
            end
        else
            eta = eta*1.1
            costheta = dot(new_params-params, grad)/(norm(new_params-params)*norm(grad))

            params = new_params
            cost = new_cost
            grad = new_grad
            hess = new_hess
        end

        if verbose
            if rem(i, verbose_every)==0
                @printf "%d: eta=%g cost=%g jtype=%s costheta=%.3f ps=" i eta cost jumptype costheta
                print_vector_g(vector_wrap(bbox, args, params))
                @printf "\n"
                if verbose_level >= 3
                    @printf "    At this point, grad is ="
                    print_vector_g(grad)
                    @printf "\n"                
                end
            end
        end
    end

    trajectory = trajectory[:,1:i]; cpm_traj = cpm_traj[:,1:i]
    return vector_wrap(bbox, args, params), trajectory, cost, cpm_traj
end


In [ ]:
bbox_Hessian_keyword_minimization()

### Looking at the distribution of VR - VL

In [ ]:
ntrials = 500
proVs, antiVs = @time(run_ntrials(ntrials; plot_list=[], model_params...))

@printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/ntrials)
@printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/ntrials)

figure(1); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-0.1:0.002:0.1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-0.1:0.002:0.1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])
